# CheckList

" In order to guide test ideation, it's useful to think of CheckList as a matrix of Capabilities x Test Types.  
*Capabilities* refers to general-purpose linguistic capabilities, which manifest in one way or another in almost any NLP application.   
We suggest that anyone CheckListing a model go through *at least* the following capabilities, trying to create MFTs, INVs, and DIRs for each if possible.
1. **Vocabulary + POS:** important words or groups of words (by part-of-speech) for the task
2. **Taxonomy**: synonyms, antonyms, word categories, etc
3. **Robustness**: to typos, irrelevant additions, contractions, etc
4. **Named Entity Recognition (NER)**: person names, locations, numbers, etc
5. **Fairness**
6. **Temporal understanding**: understanding order of events and how they impact the task
7. **Negation**
8. **Coreference** 
9. **Semantic Role Labeling (SRL)**: understanding roles such as agent, object, passive/active, etc
10. **Logic**: symmetry, consistency, conjunctions, disjunctions, etc

Notice that we are framing this as very **top-down approach**: you start with a list of capabilities and try to think of what kinds of tests can be created, based on the three test types. 

**Bottom up approach**
In this approach, we look at specific examples (from the validation dataset or elsewhere) and try to generalize them into MFTs, INVs or DIRs, placing them into a specific capability.  "

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

from nltk.corpus import wordnet as wn

import checklist
import spacy
import itertools
import pandas as pd

import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
import numpy as np
import spacy
from checklist.test_suite import TestSuite
from checklist.perturb import Perturb

In [2]:
#conda install -c conda-forge jupyter_contrib_nbextensions

## Setting CheckList utilities

" spaCy is an open-source software library for advanced natural language processing: https://github.com/explosion/spaCy. spaCy comes with pretrained statistical models and word vectors, and currently supports tokenization for 60+ languages. It features state-of-the-art speed, convolutional neural network models for tagging, parsing and named entity recognition and easy deep learning integration. "

In [3]:
nlp = spacy.load('en_core_web_sm')

/Users/Marta/opt/anaconda3/lib/python3.6/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


To create the templates and use all the functionalities, we create an Editor object

In [4]:
editor = checklist.editor.Editor()
editor.tg

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Creating a new object, TestSuite, where we'll add all the custom tests, inspired by the original notebooks plus new ones

In [5]:
suite = TestSuite()

## Exploring the linguistic data and tools available to build/expand lexicons and tests

Important words or word types for the task

### HurtLex

**HurtLex**

From the paper, section 4.2 Misogyny Identification on Social Media:
_They identified the Prostitution, Female and Male Sexual Apparatus and
Physical and Mental Diversity and Disability categories as the most informative for this task._ The task they are talking about is precisely AMI

In [6]:
hurtlex=pd.read_csv('/Users/Marta/CheckList - FBK/hurtlex/lexica/EN/1.2/hurtlex_EN.tsv', sep='\t', index_col=None, header=0)

In building the lexica, we filter out per POS = n, a, v, av 

Selecting the relevant categories: PR (words related to prostitution), ASM (male genitalia), ASF (female genitalia), DDF (physical disabilities and diversity), DDP (cognitive disabilities and diversity), OM (Homosexuality), QAS	(with potential negative connotations), CDS	(derogatory words)

In [7]:
hurtlex_AMI_pr=hurtlex[(hurtlex.category=='pr')]
hurtlex_AMI_asm=hurtlex[(hurtlex.category=='asm')]
hurtlex_AMI_asf=hurtlex[(hurtlex.category=='asf')]
hurtlex_AMI_dis=hurtlex[(hurtlex.category=='ddf')]
hurtlex_AMI_dis=hurtlex_AMI_dis.append(hurtlex[(hurtlex.category=='ddp')])
hurtlex_AMI_om=hurtlex[(hurtlex.category=='om')]
hurtlex_AMI_off=hurtlex[(hurtlex.category=='qas')]
hurtlex_AMI_off=hurtlex_AMI_off.append([(hurtlex.category=='cds')])

In [8]:
hurtlex_AMI_pr.head()

,id,pos,category,stereotype,lemma,level
26,EN940,n,pr,no,rentboy,conservative
54,EN2962,n,pr,no,courtisanerie,inclusive
64,EN1929,n,pr,no,beyotch,conservative
68,EN899,n,pr,no,sluttish,conservative
95,EN6018,n,pr,no,society figure,inclusive


In [9]:
hurtlex_AMI_asm.head()

,id,pos,category,stereotype,lemma,level
18,EN523,n,asm,no,putz,conservative
31,EN2677,n,asm,no,wankiest,conservative
70,EN1916,n,asm,no,half-wit,inclusive
97,EN861,n,asm,no,mark,conservative
152,EN2885,v,asm,no,barrack,inclusive


In [10]:
hurtlex_AMI_asf.head()

,id,pos,category,stereotype,lemma,level
51,EN337,n,asf,no,folderol,conservative
154,EN1925,n,asf,no,coo-yon,conservative
244,EN1913,n,asf,no,stupidhead,conservative
269,EN4197,n,asf,no,muff,inclusive
377,EN334,n,asf,no,trumpery,conservative


In [11]:
hurtlex_AMI_dis.head()

,id,pos,category,stereotype,lemma,level
479,EN7294,a,ddf,yes,dissatisfactory,conservative
791,EN4811,n,ddf,yes,disablement,inclusive
968,EN1784,a,ddf,yes,slimy,conservative
1107,EN4802,n,ddf,yes,differently-abled,inclusive
1803,EN4779,n,ddf,yes,handycapped,inclusive


In [12]:
hurtlex_AMI_om.head()

,id,pos,category,stereotype,lemma,level
6,EN204,n,om,no,buttfucker,inclusive
8,EN206,n,om,no,assplay,inclusive
21,EN3241,n,om,no,retrolateral,inclusive
35,EN2454,n,om,no,homophobic slurs,conservative
72,EN3234,n,om,no,anatomical term of location,inclusive


### Expanding the lexicons

Exploring the lexicons.keys

In [13]:
print(list(editor.lexicons.keys()))

['male', 'female', 'first_name', 'first_pronoun', 'last_name', 'country', 'nationality', 'city', 'religion', 'religion_adj', 'sexual_adj', 'country_city', 'male_from', 'female_from', 'last_from']


_Names_

Some of the offensive terms taken from: 
* https://www.macmillandictionary.com/thesaurus-category/british/insulting-words-for-a-woman
* http://www.slate.com/blogs/lexicon_valley/2013/09/11/top_swear_words_most_popular_curse_words_on_facebook.html
* https://github.com/minerva-ml/open-solution-toxic-comments/blob/master/external_data/compiled_bad_words.txt
* https://github.com/RobertJGabriel/Google-profanity-words/blob/master/list.txt

In [14]:
# kind of neutral, taken by identity terms' file within AMI2020's data
women_noun = ['woman', 'sister', 'daughter', 'girl', 'wife', 'young lady', 'lady',
              'grandmother', 'mum', 'girlfriend', 'partner']#, 'bride', 'housewife', Mrs', 'Miss']

# We don't need plural in templates for now
#women_noun_plural = ['women', 'sisters', 'aunts', 'daughters', 'girls', 'mothers', 'wives', 'young ladies', 'ladies', 'brides', 
#              'grandmothers', 'mums', 'housewives', 'females', 'girlfriends', 'they']

# offensive 
offensive_women_noun = ['slut', 'bitch', 'whore', 'cunt', 'spinster', 'ditsy', 'hussy', 'frigid', 
                        'witch', 'wench', 'harpy', 'trollop', 'dyke', 'skank', 'butch']

# We don't need plural in templates for now
#offensive_women_noun_plural = ['sluts', 'bitches', 'whores', 'cunts']

offensive_homosexuals = []
for term in hurtlex_AMI_om[(hurtlex_AMI_om.pos=='n')&(hurtlex_AMI_om.level=='conservative')].lemma:
    offensive_homosexuals.append(term)

offensive_generic = ['twat', 'prick', 'motherfucker', 'shit', 'shithead', 'dick', 'dickhead', 
                     'cock', 'cockface', 'cockhead', 'cocksucker', 'pussy', 'asshole', 'arsehole', 'assfucker', 
                     'fag', 'faggot', 'bastard', 'douche', 'bugger']

In [15]:
dis = ['weirdo', 'lame', 'cripple', 'invalid', 'inferm'] #'disabled', 'handicapped'
homeless = ['tramp', 'hobo', 'drifter', 'beggar', 'vagrant']#'roofless', 'homeless', 'houseless'
old = ['codger', 'grandpa', 'oldie', 'geezer', 'elder']

In [16]:
work_role = ['astronaut', 'academic', 'accountant', 'activist', 'administrator', 'advisor', 'agent', 'analyst','architect', 
             'assistant', 'attorney', 'artist' 'boxer', 'builder', 'businessman', 'cameraman', 'carpenter', 'clown', 
             'consultant', 'dentist', 'detective', 'developer', 'doctor', 'doorman', 'driver', 'electrician', 
             'economist', 'editor', 'educator', 'entrepreneur', 'executive',
             'engineer', 'farmer',  'fighter', 'fireman', 'footballer', 'foreman', 
             'historian', 'hitter', 'intern', 'interpreter', 'investigator', 'investor', 'gardener', 'housekeeper', 
             'journalist', 'librarian', 'lifeguard', 'magician', 'mechanic', 'nun', 'nurse', 'painter', 'paramedic', 
             'photographer', 'pilot', 'police', 'poet', 'postman', 'priest', 'professor', 'ranger', 'repairman', 'reporter', 
             'salesman', 'scientist', 'secretary', 'singer', 'soldier', 'student', 'surgeon', 'teacher', 'waiter', 'writer', 
             'attendant', 'officer', 'player', 'organizer', 'quarterback', 'shooter']

# These sterotyped lists are taken from the file male_occupations.txt and female_occupations.txt from the WiNo dataset 
fem_work_role = ['attendant', 'cashier', 'teacher', 'nurse', 'assistant', 'secretary', 'auditor', 'cleaner', 
                 'receptionist', 'clerk', 'counselor', 'designer', 'hairdresser', 'writer', 'housekeeper', 
                 'baker', 'accountant', 'editor', 'librarian', 'tailor']

male_work_role = ['driver', 'supervisor', 'janitor', 'cook', 'mover', 'laborer', 'construction worker',
                  'chief', 'developer', 'carpenter', 'manager', 'lawyer', 'farmer', 'salesperson', 'physician',
                  'guard', 'analyst', 'mechanic', 'sheriff', 'CEO']

For now we decide to not expand too much the lexicons, so we don't use Hurtlex's nouns

In [17]:
'''for item in hurtlex_AMI_pr[(hurtlex_AMI_pr.pos=='n')&(hurtlex_AMI_pr.level=='conservative')].lemma:
    offensive_women_noun.append(item)
offensive_women_noun'''

"for item in hurtlex_AMI_pr[(hurtlex_AMI_pr.pos=='n')&(hurtlex_AMI_pr.level=='conservative')].lemma:\n    offensive_women_noun.append(item)\noffensive_women_noun"

In [18]:
'''for item in hurtlex_AMI_dis[(hurtlex_AMI_dis.pos=='n')&(hurtlex_AMI_dis.level=='conservative')].lemma:
    offensive_generic.append(item)'''

"for item in hurtlex_AMI_dis[(hurtlex_AMI_dis.pos=='n')&(hurtlex_AMI_dis.level=='conservative')].lemma:\n    offensive_generic.append(item)"

In [19]:
editor.add_lexicon('women_noun', women_noun, overwrite=True)
editor.add_lexicon('offensive_women_noun', offensive_women_noun, overwrite=True)
editor.add_lexicon('offensive_homosexuals', offensive_homosexuals, overwrite=True)
editor.add_lexicon('offensive_generic', offensive_generic, overwrite=True)
editor.add_lexicon('dis', dis, overwrite=True)
editor.add_lexicon('homeless', homeless, overwrite=True)
editor.add_lexicon('old', old, overwrite=True)
editor.add_lexicon('work_role', work_role, overwrite=True)
editor.add_lexicon('fem_work_role', fem_work_role, overwrite=True)
editor.add_lexicon('male_work_role', male_work_role, overwrite=True)

_Adjectives_

In [20]:
print(', '.join(editor.suggest('She is {a:mask} {women_noun}.')[:100]))

amazing, extraordinary, old, incredible, older, excellent, exceptional, important, exemplary, American, outstanding, interesting, awesome, ordinary, unusual, ideal, great, honest, awful, beautiful, only, evil, lovely, emotional, odd, average, unbelievable, angry, Italian, unmarried, outspoken, elderly, elegant, admirable, good, inspiring, influential, honorable, elder, impressive, ambitious, wonderful, independent, active, enormous, eccentric, English, Indian, innocent, aging, Irish, unhappy, experienced, imposing, ancient, ugly, nice, understanding, African, artistic, unconventional, adventurous, young, strong, fantastic, happy, real, intelligent, terrific, accomplished, honorary, attractive, sweet, successful, educated, tough, actual, astonishing, absolute, uncommon, unexpected, inspirational, exquisite, remarkable, eminent, open, unfortunate, aggressive, alcoholic, imperfect, illustrious, absent, unforgettable, brilliant, unlikely, easy, astounding, invisible, iconic, single


In [21]:
print(', '.join(editor.suggest('She is {a:mask} {offensive_women_noun}.')[:100]))

fucking, evil, absolute, little, true, old, beautiful, ugly, actual, amazing, awful, excellent, incredible, arrogant, angry, enormous, utter, awesome, common, emotional, real, incompetent, abusive, insane, ignorant, American, adorable, obvious, idiot, wicked, ancient, good, great, total, attractive, pretty, adult, honest, complete, elderly, anonymous, bad, eternal, terrible, black, white, powerful, horrible, big, dead, nasty, dirty, perfect, young, damn, proud, annoying, strange, asshole, insecure, alcoholic, ultimate, extreme, immature, aggressive, animal, average, appalling, educated, older, easy, infamous, accomplished, adolescent, exquisite, odd, expert, Italian, Irish, innocent, intellectual, dangerous, active, vicious, aging, arch, anarchist, effective, female, master, original, mad, former, walking, lovely, dark, murderous, bloody, sex, blue


Some of the offensive adj are taken from:
* theodysseyonline.com/16-derogatory-words-used-to-describe-women

In [22]:
pos_adj = [                                                                             
    'adorable', 'amazing', 'astonishing', 'attractive', 'awesome', 'beautiful',
    'brilliant', 'caring', 'committed', 'cool', 'enthusiastic', 'excellent', 'exceptional',
    'extraordinary', 'fantastic', 'fun', 'good', 'great', 'happy', 'honest', 'impressive', 'incredible',
    'inspiring', 'intelligent', 'lovely', 'nice', 
    'successful', 'sweet', 'trustworthy', 'wise', 'wonderful']

neg_adj = [
    'absolute', 'aggressive', 'angry', 'annoying', 'arrogant', 
    'awful', 'bad', 'bitchy', 'bloody', 'boring', 'bossy', 'creepy', 'cruel', 
    'damn', 'dead', 'depressing', 'difficult', 'dirty', 'disappointing', 'disgusting', 
    'dreadful', 'emotional', 'evil', 'fake', 'fat', 'feisty', 'frustrating', 'fucking', 
    'giant', 'hormonal', 'horrible', 'hysterical', 'illogical', 'incompetent', 
    'irrational', 'irritating', 'lame', 'lousy', 'lying', 'nasty', 'poor', 
    'rough', 'sad', 'sassy', 'shameless', 'shitty', 'stupid', 'terrible', 'terrific', 
    'twatty', 'ugly', 'unhappy', 'wanky', 'weak', 'weird', 'whining', 'wicked', 'worst']

neutral_adj = [                                           
    'american', 'armenian','atheist', 'australian', 'black', 'british', 'christian', 'commercial', 
    'english', 'gay', 'immigrant', 'independent', 'independent', 'indian', 'indian', 'intellectual','busy', 
    'international', 'israeli', 'italian', 'jewish', 'married', 'muslim', 'old', 'poor',
    'private', 'rich', 'russian', 'single', 'tall', 'unmarried', 'white', 'working', 'young']

editor.add_lexicon('pos_adj', pos_adj, overwrite=True)
editor.add_lexicon('neg_adj', neg_adj, overwrite=True )
editor.add_lexicon('neutral_adj', neutral_adj, overwrite=True)

_Verbs_

In [23]:
print(', '.join(editor.suggest('I really {mask} {women_noun}.')[:100]))

like, miss, liked, love, mean, missed, hate, do, loved, am, appreciate, admire, respect, enjoy, did, enjoyed, dislike, want, wanted, feel, need, dig, was, care, believe, prefer, appreciated, think, have, felt, meant, know, hated, thought, see, hurt, admired, adore, needed, understand, value, got, disliked, trust, despise, thank, wish, hope, tried, fancy, the, heart, cherish, fell, married, lost, my, underestimated, blame, remember, envy, valued, resent, consider, fucking, get, a, sorry, helped, help, forgot, that, respected, cherished, say, call, had, support, called, understood, met, believed, preferred, are, likes, is, regret, trusted, wanna, missing, should, suck, worry, will, said, considered, try, liking, must, adopted


In [24]:
print(', '.join(editor.suggest('I really {mask} {offensive_women_noun}.')[:100]))

like, hate, liked, do, mean, dislike, love, am, did, miss, hated, enjoy, feel, want, was, despise, dig, prefer, enjoyed, fucking, wanted, appreciate, got, felt, disliked, think, need, wanna, get, a, meant, believe, loved, the, is, rather, missed, say, can, respect, resent, admire, understand, really, just, thought, fancy, suck, needed, know, preferred, have, hope, tried, blame, likes, go, LIKE, should, could, will, no, would, must, are, play, gotta, appreciated, adore, quite, bad, played, heart, said, don, lost, cannot, see, my, that, dont, read, trust, try, use, hurt, became, hit, dug, LOVE, fear, care, considered, smell, value, wish, fuck, regret, Like, never


In [25]:
pos_verb_present = ['like', 'enjoy', 'appreciate', 'love', 'admire',
                   'respect', 'adore', 'support', 'care for', 'fancy', 'treasure', 'trust']

neg_verb_present = ['hate', 'dislike', 'regret', 'dread', 'despise', 'blame', 'hurt', 'envy', 'pity']

neutral_verb_present = ['see', 'find', 'miss', 'understand', 'believe', 'remember', 'talk to']

pos_verb_past = ['liked', 'enjoyed', 'appreciated', 'loved', 'admired', 
                 'respected', 'adored', 'supported', 'cared for', 'treasured', 'trusted']

neg_verb_past = ['hated', 'disliked', 'regretted', 'dreaded', 'despised','blamed', 'hurt', 'envied', 'pitied']

neutral_verb_past = ['saw', 'found', 'missed', 'understood', 'believed', 'remembered', 'talked to']

editor.add_lexicon('pos_verb_present', pos_verb_present, overwrite=True)
editor.add_lexicon('neg_verb_present', neg_verb_present, overwrite=True)
editor.add_lexicon('neutral_verb_present', neutral_verb_present, overwrite=True)
editor.add_lexicon('pos_verb_past', pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb_past', neg_verb_past, overwrite=True)
editor.add_lexicon('neutral_verb_past', neutral_verb_past, overwrite=True)
editor.add_lexicon('pos_verb', pos_verb_present+ pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb', neg_verb_present + neg_verb_past, overwrite=True)
editor.add_lexicon('neutral_verb', neutral_verb_present + neutral_verb_past, overwrite=True)

_Intensifiers and reducers_

In [26]:
print(' , '.join(editor.suggest('{it} {be} {a:mask} {pos_adj} {women_noun}.', it=['She'], be=['is', 'was'])[:50]))

very , extremely , really , incredibly , truly , most , absolutely , quite , extraordinarily , amazingly , wonderfully , utterly , rather , exceptionally , especially , enormously , equally , exceedingly , unbelievably , immensely , unusually , altogether , entirely , fairly , obviously , undeniably , pretty , always , extraordinary , unexpectedly , genuinely , almost , totally , remarkably , intensely , tremendously , amazing , awfully , insanely , otherwise , just , simply , incredible , overall , highly , undoubtedly , emotionally , deeply , exceptional , appropriately


In [27]:
print(' , '.join(editor.suggest('{it} {be} {a:mask} {neg_adj} {offensive_women_noun}.', it=['She'], be=['is', 'was'])[:50]))

very , extremely , incredibly , fucking , really , rather , absolutely , pretty , especially , utterly , exceptionally , extraordinarily , exceedingly , equally , awfully , unbelievably , amazingly , bloody , enormously , immensely , damn , awful , absolute , unusually , intensely , increasingly , obviously , insanely , entirely , ugly , old , undeniably , unnecessarily , almost , real , big , endlessly , total , inherently , enormous , emotionally , overall , appropriately , even , excessively , otherwise , actual , complete , admittedly , extraordinary


In [28]:
intens_adj = ['very', 'really', 'absolutely', 'truly', 'extremely', 'quite', 'incredibly', 'especially',
              'exceptionally', 'utterly', 'rather', 'totally', 'particularly',
              'remarkably', 'pretty', 'wonderfully', 'completely',
              'entirely', 'undeniably', 'highly']

editor.add_lexicon('intens_adj', intens_adj, overwrite=True)

In [29]:
print(', '.join(editor.suggest('{i} {mask} {pos_verb} {the} {women_noun}.', i=['I', 'We', 'You'], the=['this', 'that', 'the'])[:100]))

really, always, just, also, truly, all, certainly, both, definitely, so, never, still, absolutely, totally, actually, greatly, quite, obviously, simply, especially, completely, rather, particularly, even, clearly, only, already, sure, genuinely, deeply, have, too, thoroughly, much, highly, seriously, very, had, sincerely, immediately, do, strongly, fully, desperately, generally, finally, often, most, did, usually, probably, kinda, almost, honestly, mostly, personally, guys, more, would, REALLY, surely, are, will, were, first, better, now, dearly, two, should, well, we, must, ever, instantly, each, can, secretly, might, fucking, felt, immensely, somewhat, sorely, could, may, ALL, ultimately, then, tremendously, gotta, profoundly, not, extremely, once, hardly, long, utterly, basically, hugely


In [30]:
print(', '.join(editor.suggest('{i} {mask} {neg_verb} {the} {offensive_women_noun}.', i=['I', 'We', 'You'], the=['this', 'that', 'the'])[:100]))

really, all, just, always, still, also, absolutely, actually, so, totally, already, certainly, definitely, even, almost, truly, fucking, probably, rather, simply, seriously, obviously, both, clearly, never, completely, honestly, especially, quite, sure, too, literally, basically, kinda, mostly, personally, immediately, now, only, particularly, sincerely, greatly, nearly, deeply, secretly, should, genuinely, often, thoroughly, dearly, do, generally, must, guys, much, have, practically, instantly, openly, had, will, would, finally, collectively, utterly, did, can, bitterly, very, forever, strongly, constantly, sometimes, badly, could, silently, shall, gotta, once, then, might, usually, ever, ALL, desperately, may, rightly, REALLY, fully, surely, therefore, officially, boys, most, somehow, positively, wanna, bloody, sorely, freaking


In [31]:
intens_verb = ['really', 'absolutely', 'truly', 'extremely',  'especially',  'utterly',  'totally', 'particularly', 
               'highly', 'definitely', 'certainly', 'honestly', 'strongly', 'sincerely']
reducer_adj = ['somewhat', 'kinda', 'mostly', 'probably', 'generally', 'a little', 'a bit', 'slightly']

editor.add_lexicon('intens_verb', intens_verb, overwrite=True)
editor.add_lexicon('reducer_adj', reducer_adj, overwrite=True)

Original and new lexicons keys

In [32]:
print(list(editor.lexicons.keys()))

['male', 'female', 'first_name', 'first_pronoun', 'last_name', 'country', 'nationality', 'city', 'religion', 'religion_adj', 'sexual_adj', 'country_city', 'male_from', 'female_from', 'last_from', 'women_noun', 'offensive_women_noun', 'offensive_homosexuals', 'offensive_generic', 'dis', 'homeless', 'old', 'work_role', 'fem_work_role', 'male_work_role', 'pos_adj', 'neg_adj', 'neutral_adj', 'pos_verb_present', 'neg_verb_present', 'neutral_verb_present', 'pos_verb_past', 'neg_verb_past', 'neutral_verb_past', 'pos_verb', 'neg_verb', 'neutral_verb', 'intens_adj', 'intens_verb', 'reducer_adj']


# Custom Suite, on Automatic Misogyny Identification

## Capability 1: Vocabulary + POS

### Minimal Functionality Test

Adding simple tests containing individual words (positive, negative or neutral)

_Positive_

In [33]:
test = MFT(pos_adj + pos_verb_present + pos_verb_past, labels=2)
suite.add(test, 'single positive words', 'Vocabulary', 'Simple tests involving positive words')

In [34]:
test.data

['adorable',
 'amazing',
 'astonishing',
 'attractive',
 'awesome',
 'beautiful',
 'brilliant',
 'caring',
 'committed',
 'cool',
 'enthusiastic',
 'excellent',
 'exceptional',
 'extraordinary',
 'fantastic',
 'fun',
 'good',
 'great',
 'happy',
 'honest',
 'impressive',
 'incredible',
 'inspiring',
 'intelligent',
 'lovely',
 'nice',
 'successful',
 'sweet',
 'trustworthy',
 'wise',
 'wonderful',
 'like',
 'enjoy',
 'appreciate',
 'love',
 'admire',
 'respect',
 'adore',
 'support',
 'care for',
 'fancy',
 'treasure',
 'trust',
 'liked',
 'enjoyed',
 'appreciated',
 'loved',
 'admired',
 'respected',
 'adored',
 'supported',
 'cared for',
 'treasured',
 'trusted']

_Negative_

In [35]:
test = MFT(neg_adj + neg_verb_present + neg_verb_past + offensive_women_noun + offensive_homosexuals, labels=0)
suite.add(test, 'single negative words', 'Vocabulary', 'Simple tests involving negative words')

In [36]:
test.data

['absolute',
 'aggressive',
 'angry',
 'annoying',
 'arrogant',
 'awful',
 'bad',
 'bitchy',
 'bloody',
 'boring',
 'bossy',
 'creepy',
 'cruel',
 'damn',
 'dead',
 'depressing',
 'difficult',
 'dirty',
 'disappointing',
 'disgusting',
 'dreadful',
 'emotional',
 'evil',
 'fake',
 'fat',
 'feisty',
 'frustrating',
 'fucking',
 'giant',
 'hormonal',
 'horrible',
 'hysterical',
 'illogical',
 'incompetent',
 'irrational',
 'irritating',
 'lame',
 'lousy',
 'lying',
 'nasty',
 'poor',
 'rough',
 'sad',
 'sassy',
 'shameless',
 'shitty',
 'stupid',
 'terrible',
 'terrific',
 'twatty',
 'ugly',
 'unhappy',
 'wanky',
 'weak',
 'weird',
 'whining',
 'wicked',
 'worst',
 'hate',
 'dislike',
 'regret',
 'dread',
 'despise',
 'blame',
 'hurt',
 'envy',
 'pity',
 'hated',
 'disliked',
 'regretted',
 'dreaded',
 'despised',
 'blamed',
 'hurt',
 'envied',
 'pitied',
 'slut',
 'bitch',
 'whore',
 'cunt',
 'spinster',
 'ditsy',
 'hussy',
 'frigid',
 'witch',
 'wench',
 'harpy',
 'trollop',
 'dyke',
 

_Neutral_ i.e. not-Hateful, i.e. labels=2 (positive)

In [37]:
test = MFT(neutral_adj + neutral_verb_present + neutral_verb_past, labels=2)
suite.add(test, 'single neutral words', 'Vocabulary', 'Simple tests involving neutral words')

In [38]:
test.data

['american',
 'armenian',
 'atheist',
 'australian',
 'black',
 'british',
 'christian',
 'commercial',
 'english',
 'gay',
 'immigrant',
 'independent',
 'independent',
 'indian',
 'indian',
 'intellectual',
 'busy',
 'international',
 'israeli',
 'italian',
 'jewish',
 'married',
 'muslim',
 'old',
 'poor',
 'private',
 'rich',
 'russian',
 'single',
 'tall',
 'unmarried',
 'white',
 'working',
 'young',
 'see',
 'find',
 'miss',
 'understand',
 'believe',
 'remember',
 'talk to',
 'saw',
 'found',
 'missed',
 'understood',
 'believed',
 'remembered',
 'talked to']

Words in context: creating data-test from templates

_Positive and negative_

In [39]:
t = editor.template('{it} {women_noun} {be} {pos_adj}.', it=['The', 'That'], be=['is', 'was'], labels=2, nsamples=200, save=True)
t += editor.template('{it} {be} {a:pos_adj} {women_noun}.', it=['She'], be=['is', 'was'], labels=2, nsamples=200, save=True)
t += editor.template('{i} {pos_verb} {the} {women_noun}.', i=['I', 'We', 'You'], the=['this', 'that'], labels=2, nsamples=200, save=True)

t += editor.template('{it} {offensive_women_noun} {be} {neg_adj}.', it=['The', 'That'], be=['is', 'was'], labels=0, nsamples=200, save=True)
t += editor.template('{it} {be} {a:neg_adj} {offensive_women_noun}.', it=['She'], be=['is', 'was'], labels=0, nsamples=200, save=True)
t += editor.template('{i} {neg_verb} {the} {offensive_women_noun}.', i=['I', 'We', 'You'], the=['this', 'that'], labels=0, nsamples=200, save=True)

test = MFT(**t)
suite.add(test, 'sentiment-laden words in context', 'Vocabulary', 'Use positive and negative verbs and adjectives with common women nouns')

In [40]:
test.data

['The sister is caring.',
 'The mum is good.',
 'That mum was beautiful.',
 'That girl was astonishing.',
 'That partner was astonishing.',
 'That girlfriend is trustworthy.',
 'That mum is fantastic.',
 'That grandmother was excellent.',
 'That girl is successful.',
 'The lady was enthusiastic.',
 'The young lady is successful.',
 'The sister is committed.',
 'The girlfriend was incredible.',
 'The girl was successful.',
 'The wife was good.',
 'The mum is trustworthy.',
 'That sister was beautiful.',
 'The partner is cool.',
 'The wife is extraordinary.',
 'The woman is committed.',
 'The girl was enthusiastic.',
 'The girlfriend was excellent.',
 'That daughter was inspiring.',
 'That woman was brilliant.',
 'That wife is adorable.',
 'That girlfriend was fantastic.',
 'The daughter is awesome.',
 'The grandmother was intelligent.',
 'The grandmother is excellent.',
 'That partner is attractive.',
 'The girlfriend is sweet.',
 'That mum is adorable.',
 'The lady was fantastic.',
 'T

_Neutral_ i.e. not-Hateful, i.e. labels=2 (positive)

In [41]:
t = editor.template('{it} {women_noun} {be} {neutral_adj}.', it=['The', 'That'], be=['is', 'was'], labels=2,  nsamples=200, save=True)
t += editor.template('{it} {be} {a:neutral_adj} {women_noun}.', it=['She'], be=['is', 'was'], labels=2,  nsamples=200, save=True)
t += editor.template('{i} {neutral_verb} {the} {women_noun}.', i=['I', 'We', 'You'], the=['this', 'that'], labels=2,  nsamples=200, save=True)
test = MFT(t.data, labels=2, templates=t.templates)
suite.add(test, 'neutral words in context', 'Vocabulary', 'Use neutral verbs and adjectives with common women nouns')

In [42]:
test.data

['The partner is christian.',
 'That mum was independent.',
 'That partner is unmarried.',
 'The young lady was australian.',
 'The woman was intellectual.',
 'That young lady is commercial.',
 'That wife is australian.',
 'That grandmother was private.',
 'That wife is independent.',
 'The girl is immigrant.',
 'That sister was independent.',
 'That daughter is british.',
 'The grandmother was young.',
 'That woman is single.',
 'The girl is married.',
 'That young lady was married.',
 'The girlfriend is independent.',
 'That young lady was australian.',
 'The girl was jewish.',
 'That girl is christian.',
 'That daughter is armenian.',
 'That girlfriend is armenian.',
 'That daughter was working.',
 'That young lady is israeli.',
 'The girlfriend is commercial.',
 'That girlfriend is christian.',
 'The wife was private.',
 'The mum was commercial.',
 'That grandmother is unmarried.',
 'That young lady is independent.',
 'The wife is independent.',
 'That girlfriend was intellectual.'

### Intensifiers and reducers

In [43]:
# We're  expecting the score to raise 
monotonic_label = Expect.monotonic(increasing=True, tolerance=0.1)
pos_pred = lambda pred, *args, **kwargs: pred == 2
monotonic_label = Expect.slice_pairwise(monotonic_label, pos_pred)

In [44]:
t = editor.template(['{it} {be} {a:pos_adj} {women_noun}.', '{it} {be} {a:intens} {pos_adj} {women_noun}.'] , intens=intens_adj, it=['She'], be=['is', 'was'], nsamples=200, labels=2, save=True)
t += editor.template(['{i} {pos_verb} {the} {women_noun}.', '{i} {intens} {pos_verb} {the} {women_noun}.'], intens=intens_verb, i=['I', 'We', 'You'], the=['this', 'that'], nsamples=200, labels=2, save=True)

t.data

[['She is a honest woman.', 'She is an absolutely honest woman.'],
 ['She is an inspiring girlfriend.', 'She is a really inspiring girlfriend.'],
 ['She is an impressive mum.', 'She is an exceptionally impressive mum.'],
 ['She was an impressive girl.', 'She was a truly impressive girl.'],
 ['She is a committed lady.', 'She is an extremely committed lady.'],
 ['She is a wise girl.', 'She is an incredibly wise girl.'],
 ['She is an adorable daughter.', 'She is an especially adorable daughter.'],
 ['She was a trustworthy wife.', 'She was an especially trustworthy wife.'],
 ['She is a successful girlfriend.', 'She is a truly successful girlfriend.'],
 ['She is a happy partner.', 'She is an utterly happy partner.'],
 ['She was an amazing lady.', 'She was a remarkably amazing lady.'],
 ['She is a beautiful woman.', 'She is a really beautiful woman.'],
 ['She was a lovely partner.', 'She was an extremely lovely partner.'],
 ['She was a happy partner.', 'She was a rather happy partner.'],
 ['

In [45]:
test = DIR(t.data, monotonic_label, templates=t.templates)

description = '''Test is composed of pairs of sentences (x1, x2), where we add an intensifier
such as "really",or "very" to x2 and expect the confidence to NOT go down (with tolerance=0.1). e.g.:
x1 = "She was a good mother"
x2 = "She was a very good mother"
We disregard cases where the prediction of x1 is neutral.
'''

suite.add(test, 'intensifiers for positive samples', 'Vocabulary', description)

In [46]:
# We're  expecting the score to fall 
monotonic_label_down = Expect.monotonic(increasing=False, tolerance=0.1)
neg_pred = lambda pred, *args, **kwargs: pred == 0
monotonic_label_down = Expect.slice_pairwise(monotonic_label_down, neg_pred)

In [47]:
t = editor.template(['{it} {be} {a:neg_adj} {offensive_women_noun}.', '{it} {be} {a:intens} {neg_adj} {offensive_women_noun}.'] , intens=intens_adj, it=['She'], be=['is', 'was'], nsamples=200, labels=0, save=True)
t += editor.template(['{i} {neg_verb} {the} {offensive_women_noun}.', '{i} {intens} {neg_verb} {the} {offensive_women_noun}.'], intens=intens_verb, i=['I', 'We', 'You'], the=['this', 'that'], nsamples=200, labels=0, save=True)
t += editor.template(['{it} {women_noun} {be} {neg_adj}.', '{it} {women_noun} {be} {red} {neg_adj}.'] , red=reducer_adj, it=['The', 'That'], be=['is', 'was'], nsamples=200, labels=0, save=True)
t.data

[['She was a fucking trollop.', 'She was an especially fucking trollop.'],
 ['She was a terrible skank.', 'She was a very terrible skank.'],
 ['She was a stupid cunt.', 'She was a quite stupid cunt.'],
 ['She was an ugly wench.', 'She was an incredibly ugly wench.'],
 ['She is an irrational skank.', 'She is a completely irrational skank.'],
 ['She was a hysterical hussy.', 'She was a pretty hysterical hussy.'],
 ['She was a depressing hussy.', 'She was a particularly depressing hussy.'],
 ['She is an incompetent trollop.',
  'She is an incredibly incompetent trollop.'],
 ['She is a terrible wench.', 'She is an entirely terrible wench.'],
 ['She is a bossy slut.', 'She is an extremely bossy slut.'],
 ['She is an irrational bitch.', 'She is an exceptionally irrational bitch.'],
 ['She is a bad frigid.', 'She is an absolutely bad frigid.'],
 ['She is an awful frigid.', 'She is a highly awful frigid.'],
 ['She was a fake ditsy.', 'She was a very fake ditsy.'],
 ['She is a terrific bitch.',

In [48]:
test = DIR(t.data, monotonic_label_down, templates=t.templates)

description = '''Test is composed of pairs of sentences (x1, x2), where we add a reducer
such as "somewhat", or "kinda" to x2 and expect the confidence to NOT go up (with tolerance=0.1). e.g.:
x1 = "The mum was good."
x2 = "The mum was somewhat good."
'''

suite.add(test, 'intensifiers and reducers for negative samples', 'Vocabulary', description)

### INVariance: change neutral words

In [49]:
neutral_words = set(
    ['.', 'the', 'The', ',', 'a', 'A', 'and', 'of', 'to', 'it', 'that', 'in',
     'this', 'for',  'you', 'there', 'or', 'an', 'by', 'about', 'my',
     'in', 'of', 'have', 'with', 'was', 'at', 'it', 'get', 'from', 'this'
    ])

forbidden = set(['No', 'no', 'Not', 'not', 'Nothing', 'nothing', 'without', 'but'] + pos_adj + neg_adj + pos_verb_present + pos_verb_past + neg_verb_present + neg_verb_past)

def change_neutral(d):
#     return d.text
    examples = []
    subs = []
    words_in = [x for x in d.capitalize().split() if x in neutral_words]
    if not words_in:
        return None
    for w in words_in:
        suggestions = [x for x in editor.suggest_replace(d, w, beam_size=5, words_and_sentences=True) if x[0] not in forbidden]
        examples.extend([x[1] for x in suggestions])
        subs.extend(['%s -> %s' % (w, x[0]) for x in suggestions])
    if examples:
        idxs = np.random.choice(len(examples), min(len(examples), 10), replace=False)
        return [examples[i] for i in idxs]#, [subs[i] for i in idxs])
# Perturb.perturb(parsed_data[:5], perturb)

In [50]:
t_pert = editor.template('{it} {women_noun} {be} {pos_adj}.', it=['The', 'That'], be=['is', 'was'], labels=2, nsamples=10, save=True)
t_pert += editor.template('{it} {be} {a:pos_adj} {women_noun}.', it=['She'], be=['is', 'was'], labels=2, nsamples=10, save=True)
t_pert += editor.template('{i} {pos_verb} {the} {women_noun}.', i=['I', 'We', 'You'], the=['this', 'that'], labels=2, nsamples=10, save=True)
t_pert += editor.template('{it} {offensive_women_noun} {be} {neg_adj}.', it=['The', 'That'], be=['is', 'was'], labels=0, nsamples=10, save=True)
t_pert += editor.template('{it} {be} {a:neg_adj} {offensive_women_noun}.', it=['She'], be=['is', 'was'], labels=0, nsamples=10, save=True)
t_pert += editor.template('{i} {neg_verb} {the} {offensive_women_noun}.', i=['I', 'We', 'You'], the=['this', 'that'], labels=0, nsamples=10, save=True)

In [51]:
t = Perturb.perturb(t_pert.data, change_neutral, nsamples=200) 

test = INV(t.data)

description = 'Change a set of neutral words with other context-appropriate neutral words (using BERT).'

suite.add(test, 'change neutral words with BERT', 'Vocabulary', description)

In [52]:
t.data

[['We fancy that mum.',
  'We fancy each mum.',
  'We fancy her mum.',
  'We fancy the mum.',
  'We fancy my mum.',
  'We fancy you mum.',
  'We fancy out mum.',
  'We fancy old mum.',
  'We fancy his mum.',
  'We fancy being mum.',
  'We fancy your mum.'],
 ['You appreciated this grandmother.',
  'You appreciated to grandmother.',
  'You appreciated their grandmother.',
  'You appreciated your grandmother.',
  'You appreciated his grandmother.',
  'You appreciated another grandmother.',
  'You appreciated that grandmother.',
  'You appreciated you grandmother.',
  'You appreciated dear grandmother.',
  'You appreciated the grandmother.',
  'You appreciated being grandmother.'],
 ['She was a nice grandmother.',
  'She is a nice grandmother.',
  'She had a nice grandmother.'],
 ['The frigid was depressing.',
  'The frigid turns depressing.',
  'The frigid The depressing.',
  'And frigid was depressing.',
  'Even frigid was depressing.',
  'The frigid turn depressing.',
  'The frigid & d

### Add negative phrases

In [53]:
positive = editor.template('I {pos_verb_present} you.').data
positive += editor.template('Really {pos_adj}!').data
negative = editor.template('I {neg_verb_present} you.').data
negative += editor.template('Sooo {neg_adj}!').data

In [54]:
def add_phrase_function(phrases):
    def pert(d):
        while d[-1].pos_ == 'PUNCT':
            d = d[:-1]
        d = d.text
        ret = [d + '. ' + x for x in phrases]
        idx = np.random.choice(len(ret), 10, replace=False)
        ret = [ret[i] for i in idx]
        return ret
    return pert

def positive_change(orig_conf, conf):
    softmax = type(orig_conf) in [np.array, np.ndarray]
    if not softmax or orig_conf.shape[0] != 3:
        raise(Exception('Need prediction function to be softmax with 3 labels (negative, neutral, positive)'))
    return orig_conf[0] - conf[0] + conf[2] - orig_conf[2]

def diff_up(orig_pred, pred, orig_conf, conf, labels=None, meta=None):
    tolerance = 0.1
    change = positive_change(orig_conf, conf)
    if change + tolerance >= 0:
        return True
    else:
        return change + tolerance
    
def diff_down(orig_pred, pred, orig_conf, conf, labels=None, meta=None):
    tolerance = 0.1
    change = positive_change(orig_conf, conf)
    if change - tolerance <= 0:
        return True
    else:
        return -(change - tolerance)
    
goes_up = Expect.pairwise(diff_up)
goes_down = Expect.pairwise(diff_down)

In [55]:
t_parsed_data = list(nlp.pipe(t_pert.data))

In [56]:
t = Perturb.perturb(t_parsed_data, add_phrase_function(positive), nsamples=200)

test = DIR(t.data, goes_up)

description = 'Add very positive phrases (e.g. I love you) to the end of sentences, expect probability of positive to NOT go down (tolerance=0.1)'

suite.add(test, 'add positive phrases', 'Vocabulary', description)

In [57]:
t.data

[['The butch is bitchy.',
  'The butch is bitchy. Really committed!',
  'The butch is bitchy. I enjoy you.',
  'The butch is bitchy. Really intelligent!',
  'The butch is bitchy. Really inspiring!',
  'The butch is bitchy. Really cool!',
  'The butch is bitchy. Really beautiful!',
  'The butch is bitchy. I respect you.',
  'The butch is bitchy. I admire you.',
  'The butch is bitchy. Really impressive!',
  'The butch is bitchy. Really brilliant!'],
 ['I regretted this witch.',
  'I regretted this witch. Really sweet!',
  'I regretted this witch. Really great!',
  'I regretted this witch. Really caring!',
  'I regretted this witch. Really happy!',
  'I regretted this witch. I adore you.',
  'I regretted this witch. Really honest!',
  'I regretted this witch. Really nice!',
  'I regretted this witch. Really wonderful!',
  'I regretted this witch. Really attractive!',
  'I regretted this witch. Really enthusiastic!'],
 ['I despised that spinster.',
  'I despised that spinster. I fancy you

In [58]:
t = Perturb.perturb(t_parsed_data, add_phrase_function(negative), nsamples=200)

test = DIR(t.data, goes_down)

description = 'Add very negative phrases (e.g. I hate you) to the end of sentences, expect probability of positive to NOT go up (tolerance=0.1)'

suite.add(test, 'add negative phrases', 'Vocabulary', description)

In [59]:
t.data

[['She is an absolute spinster.',
  'She is an absolute spinster. Sooo disappointing!',
  'She is an absolute spinster. Sooo hormonal!',
  'She is an absolute spinster. Sooo cruel!',
  'She is an absolute spinster. Sooo fucking!',
  'She is an absolute spinster. Sooo horrible!',
  'She is an absolute spinster. Sooo ugly!',
  'She is an absolute spinster. Sooo fat!',
  'She is an absolute spinster. Sooo bad!',
  'She is an absolute spinster. Sooo lying!',
  'She is an absolute spinster. Sooo arrogant!'],
 ['I despised that spinster.',
  'I despised that spinster. Sooo dreadful!',
  'I despised that spinster. Sooo depressing!',
  'I despised that spinster. Sooo nasty!',
  'I despised that spinster. Sooo irrational!',
  'I despised that spinster. Sooo cruel!',
  'I despised that spinster. Sooo weak!',
  'I despised that spinster. Sooo bad!',
  'I despised that spinster. Sooo stupid!',
  'I despised that spinster. Sooo damn!',
  'I despised that spinster. Sooo sassy!'],
 ['That girl was br

## Capability 2: Robustness

To typos, irrelevant changes, etc

### INVariance: adding irrelevant linguistic segments before and after

In [60]:
import string
def random_string(n):
    return ''.join(np.random.choice([x for x in string.ascii_letters + string.digits], n))
def random_url(n=6):
    return 'https://t.co/%s' % random_string(n)
def random_handle(n=6):
    return '@%s' % random_string(n)

def add_irrelevant(sentence):
    urls_and_handles = [random_url(n=6) for _ in range(5)] + [random_handle() for _ in range(5)]
    irrelevant_before = ['@miss '] + urls_and_handles
    irrelevant_after = urls_and_handles 
    rets = ['%s %s' % (x, sentence) for x in irrelevant_before ]
    rets += ['%s %s' % (sentence, x) for x in irrelevant_after]
    return rets

In [61]:
t = Perturb.perturb(t_pert.data, add_irrelevant, nsamples=200)

test = INV(t.data)

suite.add(test, 'add random urls and handles', 'Robustness', 'Add randomly generated urls and handles to the start or end of sentence')

In [62]:
t.data

[['She was an irritating slut.',
  '@miss  She was an irritating slut.',
  'https://t.co/CfVv10 She was an irritating slut.',
  'https://t.co/0g9AaQ She was an irritating slut.',
  'https://t.co/cxJAAb She was an irritating slut.',
  'https://t.co/z4aAin She was an irritating slut.',
  'https://t.co/5iR8HH She was an irritating slut.',
  '@5zuXDO She was an irritating slut.',
  '@x1ttUZ She was an irritating slut.',
  '@77Feaj She was an irritating slut.',
  '@iFQaYX She was an irritating slut.',
  '@eI6WKG She was an irritating slut.',
  'She was an irritating slut. https://t.co/CfVv10',
  'She was an irritating slut. https://t.co/0g9AaQ',
  'She was an irritating slut. https://t.co/cxJAAb',
  'She was an irritating slut. https://t.co/z4aAin',
  'She was an irritating slut. https://t.co/5iR8HH',
  'She was an irritating slut. @5zuXDO',
  'She was an irritating slut. @x1ttUZ',
  'She was an irritating slut. @77Feaj',
  'She was an irritating slut. @iFQaYX',
  'She was an irritating slu

### Punctuation, contractions, typos

In [63]:
t = Perturb.perturb(t_parsed_data, Perturb.punctuation, nsamples=200)

test = INV(t.data)

suite.add(test, 'punctuation', 'Robustness', 'Strip punctuation and / or add "."')

In [64]:
t.data

[['The butch is bitchy.', 'The butch is bitchy'],
 ['We pitied this slut.', 'We pitied this slut'],
 ['I regretted this witch.', 'I regretted this witch'],
 ['That ditsy was feisty.', 'That ditsy was feisty'],
 ['That woman is beautiful.', 'That woman is beautiful'],
 ['I despised that dyke.', 'I despised that dyke'],
 ['I despised that spinster.', 'I despised that spinster'],
 ['The slut was absolute.', 'The slut was absolute'],
 ['You regretted this hussy.', 'You regretted this hussy'],
 ['We adored that daughter.', 'We adored that daughter'],
 ['I despised this cunt.', 'I despised this cunt'],
 ['She was a giant cunt.', 'She was a giant cunt'],
 ['The butch was boring.', 'The butch was boring'],
 ['The wife is beautiful.', 'The wife is beautiful'],
 ['I love this grandmother.', 'I love this grandmother'],
 ['The ditsy is giant.', 'The ditsy is giant'],
 ['I trusted this girlfriend.', 'I trusted this girlfriend'],
 ['She was a disgusting spinster.', 'She was a disgusting spinster'],


In [65]:
t = Perturb.perturb(t_pert.data, Perturb.add_typos, nsamples=200, typos=1)

test = INV(t.data)

suite.add(test, 'typos', 'Robustness', 'Add one typo to input by swapping two adjacent characters')

In [66]:
t.data

[['I regretted this witch.', 'I regrettde this witch.'],
 ['The skank was ugly.', 'The skank aws ugly.'],
 ['She is a brilliant girl.', 'She isa  brilliant girl.'],
 ['The butch was boring.', 'The butch was boirng.'],
 ['She was a giant cunt.', 'She aws a giant cunt.'],
 ['I appreciate that girl.', 'I papreciate that girl.'],
 ['I dread this wench.', 'I dreda this wench.'],
 ['That young lady is exceptional.', 'That young lady i sexceptional.'],
 ['She is a disgusting frigid.', 'She is a disgustnig frigid.'],
 ['The bitch is difficult.', 'The bitch is dififcult.'],
 ['She is a weird slut.', 'Seh is a weird slut.'],
 ['I dislike that bitch.', 'I dislike taht bitch.'],
 ['She was an irritating slut.', 'Sh ewas an irritating slut.'],
 ['We regretted that slut.', 'We rergetted that slut.'],
 ['She is a happy woman.', 'She i sa happy woman.'],
 ['She was a fucking witch.', 'She was a fucknig witch.'],
 ['She is an absolute spinster.', 'She is an absoulte spinster.'],
 ['That frigid was bad.

In [67]:
t = Perturb.perturb(t_pert.data, Perturb.add_typos, nsamples=200, typos=2)

test = INV(t.data)

suite.add(test, '2 typos', 'Robustness', 'Add two typos to input by swapping two adjacent characters twice')

In [68]:
t.data

[['That partner was intelligent.', 'That patrne rwas intelligent.'],
 ['We pitied this slut.', 'We iptied htis slut.'],
 ['The spinster is poor.', 'The spinster ispo or.'],
 ['She is a brilliant girl.', 'Seh is a brillinat girl.'],
 ['That young lady was great.', 'That young lyad was great.'],
 ['That sister is good.', 'That sisetr isg ood.'],
 ['The bitch is difficult.', 'Theb itch isd ifficult.'],
 ['She was a giant cunt.', 'Shew a sa giant cunt.'],
 ['The mum is inspiring.', 'Teh mu mis inspiring.'],
 ['The ditsy is giant.', 'The idtsy is gian.t'],
 ['She was a caring wife.', 'She was  aacring wife.'],
 ['That girl was brilliant.', 'That girl was brlilian.t'],
 ['She is an absolute spinster.', 'hSe is an absoluet spinster.'],
 ['She is an exceptional daughter.', 'Sh eis ane xceptional daughter.'],
 ['She was a sweet sister.', 'She was a sweet sister.'],
 ['I regretted this witch.', 'I regretted this wict.h'],
 ['I dislike that bitch.', 'I dilsike that btich.'],
 ['The slut was absol

In [69]:
t = Perturb.perturb(t_pert.data, Perturb.contractions, nsamples=200)

test = INV(t.data)

suite.add(test, 'contractions', 'Robustness', 'Contract or expand contractions, e.g. What is -> What\'s')

In [70]:
t.data

[['She is a happy woman.', "She's a happy woman."],
 ['She is a disgusting frigid.', "She's a disgusting frigid."],
 ['She is an impressive mum.', "She's an impressive mum."],
 ['She is an adorable grandmother.', "She's an adorable grandmother."],
 ['She is a lying bitch.', "She's a lying bitch."],
 ['She is a weird slut.', "She's a weird slut."],
 ['She is a twatty harpy.', "She's a twatty harpy."],
 ['She is an absolute spinster.', "She's an absolute spinster."],
 ['She is a sweet wife.', "She's a sweet wife."],
 ['She is an exceptional daughter.', "She's an exceptional daughter."],
 ['She is a brilliant girl.', "She's a brilliant girl."]]

## Capability 3: NER

Appropriately understanding Named Entities

In [71]:
t = editor.template('I met with {first_name} {last_name} last night.', nsamples=100, save=True)
test = MFT(t.data, labels=2, templates=t.templates)
suite.add(test, 'change with English names', 'NER', 'Replace names with other common English names')

In [72]:
t.data[:5]

['I met with Carol Murray last night.',
 'I met with Maria Sullivan last night.',
 'I met with Dorothy Thomas last night.',
 'I met with Greg Thomas last night.',
 'I met with Sue Peterson last night.']

With foreign names:

In [73]:
first = [x.split()[0] for x in editor.lexicons.male_from.Germany +  editor.lexicons.female_from.Germany]
last = [x.split()[0] for x in editor.lexicons.last_from.Germany]
t = editor.template('I met with {first_name} {last_name} last night.', first_name=first, last_name=last, nsamples=100, save=True)
test = MFT(t.data, labels=2, templates=t.templates)
suite.add(test, 'change with german names', 'NER', 'Replace names with other foreign names (german)')

In [74]:
t.data[:5]

['I met with Astrid Ludwig last night.',
 'I met with Lothar Grimm last night.',
 'I met with Lisa Ackermann last night.',
 'I met with Jakob Keller last night.',
 'I met with Cornelia Martin last night.']

In [75]:
first = [x.split()[0] for x in editor.lexicons.male_from.Vietnam +  editor.lexicons.female_from.Vietnam]
last = [x.split()[0] for x in editor.lexicons.last_from.Vietnam]
t = editor.template('I met with {first_name} {last_name} last night.', first_name=first, last_name=last, nsamples=100, save=True)
test = MFT(t.data, labels=2, templates=t.templates)
suite.add(test, 'change with vietnamese names', 'NER', 'Replace names with other foreign names (vietnamese)')

In [76]:
t.data[:5]

['I met with Eugene Kelly last night.',
 'I met with Donald Diem last night.',
 'I met with John Walters last night.',
 'I met with Piotr Tiêu last night.',
 'I met with An Tạ last night.']

In [77]:
first = [x.split()[0] for x in editor.lexicons.male_from.Brazil +  editor.lexicons.female_from.Brazil]
last = [x.split()[0] for x in editor.lexicons.last_from.Brazil]
t = editor.template('I met with {first_name} {last_name} last night.', first_name=first, last_name=last, nsamples=100, save=True)
test = MFT(t.data, labels=2, templates=t.templates)
suite.add(test, 'change with brazilian names', 'NER', 'Replace names with other foreign names (brazilian)')

In [78]:
t.data[:5]

['I met with Paula Chaves last night.',
 'I met with Joaquim Menezes last night.',
 'I met with Gabriel da last night.',
 'I met with Antônio Chaves last night.',
 'I met with Giovanna Aguiar last night.']

## Capability 4: Temporal Awareness

Understanding order of events

In [79]:
change = ['but', 'even though', 'although']
t = editor.template(['I used to {neg_verb_present} this girl, {change} now I {pos_verb_present} it.',
                                 'I {pos_verb_present} this girl, {change} I used to {neg_verb_present} it.',
                                 'In the past I would {neg_verb_present} this girl, {change} now I {pos_verb} it.',
                                 'I {pos_verb_present} this girl, {change} in the past I would {neg_verb_present} it.',
                                ] ,
                                change=change, unroll=True, nsamples=50, save=True, labels=2)

t += editor.template(['I used to {pos_verb_present} this girl, {change} now I {neg_verb_present} it.',
                                 'I {neg_verb_present} this girl, {change} I used to {pos_verb_present} it.',
                                 'In the past I would {pos_verb_present} this girl, {change} now I {neg_verb_present} it.',
                                 'I {neg_verb_present} this girl, {change} in the past I would {pos_verb_present} it.',
                                ] ,
                                change=change, unroll=True, nsamples=50, save=True, labels=0)

t += editor.template(['I used to think this girl was {pos_adj}, {change} now I think she is {a:offensive_women_noun}.',
                                 'I think this girl is {a:offensive_women_noun}, {change} I used to think she was {pos_adj}.',
                                 'In the past I thought this girl was {pos_adj}, {change} now I think she is {a:offensive_women_noun}.',
                                 'I think this girl is {a:offensive_women_noun}, {change} in the past I thought she was {pos_adj}.',
                                ] ,
                                 change=change, unroll=True, nsamples=150, save=True, labels=0)

test = MFT(**t)

description = '''Have two conflicing statements, one about the past and one about the present.
Expect the present to carry the sentiment. Examples:
I used to love this *, now I hate it -> should be negative
I love this *, although I used to hate it -> should be positive
'''

suite.add(test, 'used to, but now', 'Temporal', description)

In [80]:
t.data

['I used to hurt this girl, but now I fancy it.',
 'I fancy this girl, but I used to hurt it.',
 'In the past I would hurt this girl, but now I treasure it.',
 'I fancy this girl, but in the past I would hurt it.',
 'I used to despise this girl, but now I care for it.',
 'I care for this girl, but I used to despise it.',
 'In the past I would despise this girl, but now I enjoyed it.',
 'I care for this girl, but in the past I would despise it.',
 'I used to blame this girl, although now I admire it.',
 'I admire this girl, although I used to blame it.',
 'In the past I would blame this girl, although now I appreciate it.',
 'I admire this girl, although in the past I would blame it.',
 'I used to dislike this girl, even though now I treasure it.',
 'I treasure this girl, even though I used to dislike it.',
 'In the past I would dislike this girl, even though now I treasure it.',
 'I treasure this girl, even though in the past I would dislike it.',
 'I used to despise this girl, even th

_Used to_ should reduce

In [88]:
t = editor.template(['She {be} {a:adj} {women_noun}.', 'I used to think she {be} {a:adj} {women_noun}.'], be=['is', 'was'], adj=editor.lexicons['neg_adj'], nsamples=100, save=True)
t += editor.template(['{i} {verb} {the} {women_noun}.', '{i} used to {verb} {the} {women_noun}.'], i=['I', 'We', 'You'], the=['this', 'that'], verb=editor.lexicons['neg_verb_present'], nsamples=100, save=True)

test = DIR(t.data, monotonic_label_down, templates=t.templates)

suite.add(test, '"used to" should reduce', 'Temporal', 'A model should not be more confident on "I used to think X" when compared to "X", e.g. "I used to love her" should have less confidence than "I love her"', overwrite=True)

In [89]:
t.data

[['She is an emotional woman.', 'I used to think she is an emotional woman.'],
 ['She is an emotional partner.',
  'I used to think she is an emotional partner.'],
 ['She is a hormonal girlfriend.',
  'I used to think she is a hormonal girlfriend.'],
 ['She was a bad mum.', 'I used to think she was a bad mum.'],
 ['She is an annoying mum.', 'I used to think she is an annoying mum.'],
 ['She was a rough sister.', 'I used to think she was a rough sister.'],
 ['She is a bossy woman.', 'I used to think she is a bossy woman.'],
 ['She was a disappointing sister.',
  'I used to think she was a disappointing sister.'],
 ['She is a bitchy sister.', 'I used to think she is a bitchy sister.'],
 ['She is a hysterical grandmother.',
  'I used to think she is a hysterical grandmother.'],
 ['She is an evil girlfriend.', 'I used to think she is an evil girlfriend.'],
 ['She is a wicked grandmother.',
  'I used to think she is a wicked grandmother.'],
 ['She is a disgusting woman.', 'I used to think s

_Before_ should reduce

In [90]:
t = editor.template(['She {be} {a:adj} {women_noun}.', 'Before I thought she {be} {a:adj} {women_noun}.'], be=['is', 'was'], adj=editor.lexicons['neg_adj'], nsamples=100, save=True)

test = DIR(t.data, monotonic_label_down, templates=t.templates)

suite.add(test, '"before" should reduce', 'Temporal', 'A model should not be more confident on "Before I thought X" when compared to "X"')

In [91]:
t.data

[['She is a wicked sister.', 'Before I thought she is a wicked sister.'],
 ['She is an angry mum.', 'Before I thought she is an angry mum.'],
 ['She is a lousy mum.', 'Before I thought she is a lousy mum.'],
 ['She is a hysterical daughter.',
  'Before I thought she is a hysterical daughter.'],
 ['She is a difficult sister.', 'Before I thought she is a difficult sister.'],
 ['She is a worst partner.', 'Before I thought she is a worst partner.'],
 ['She is an awful daughter.', 'Before I thought she is an awful daughter.'],
 ['She is a fucking daughter.', 'Before I thought she is a fucking daughter.'],
 ['She is a fake woman.', 'Before I thought she is a fake woman.'],
 ['She was an emotional partner.',
  'Before I thought she was an emotional partner.'],
 ['She is a creepy wife.', 'Before I thought she is a creepy wife.'],
 ['She was a horrible sister.', 'Before I thought she was a horrible sister.'],
 ['She was an illogical girlfriend.',
  'Before I thought she was an illogical girlfri


## Capability 5: Negation

Simple templates: adding negations

In [92]:
neg = ['I can\'t say I', 'I don\'t', 'I would never say I', 'I don\'t think I', 'I didn\'t' ]

t = editor.template('{it} {offensive_women_noun} {nt} {pos_adj}.', it=['This', 'That'], nt=['is not', 'isn\'t'], nsamples=100, save=True)
t += editor.template('{it} {benot} {a:pos_adj} {offensive_women_noun}.', it=['She'], benot=['is not',  'isn\'t', 'was not', 'wasn\'t'], nsamples=100, save=True)

t += editor.template('{neg} {pos_verb_present} {the} {offensive_women_noun}.', neg=neg, the=['this', 'that'], nsamples=100, save=True)
t += editor.template('{neg} {pos_verb_present} {the} {offensive_homosexuals}.', neg=neg, the=['this', 'that'], nsamples=100, save=True)

t += editor.template('No one {pos_verb_present}s {the} {offensive_women_noun}.', neg=neg, the=['this', 'that'], nsamples=100, save=True)
t += editor.template('No one {pos_verb_present}s {a:offensive_homosexuals}.', neg=neg, nsamples=100, save=True)

test = MFT(t.data, labels=0, templates=t.templates)

suite.add(test, 'simple negations: negative', 'Negation', 'Very simple negations of positive statements')

In [93]:
t.data

["This skank isn't trustworthy.",
 'This trollop is not intelligent.',
 "This hussy isn't impressive.",
 "That spinster isn't trustworthy.",
 'That frigid is not great.',
 "This cunt isn't committed.",
 "That bitch isn't enthusiastic.",
 "This witch isn't awesome.",
 'That ditsy is not cool.',
 "This frigid isn't adorable.",
 "That slut isn't sweet.",
 "This ditsy isn't nice.",
 "That harpy isn't wise.",
 'That cunt is not brilliant.',
 "This spinster isn't great.",
 "This wench isn't awesome.",
 "That trollop isn't excellent.",
 'This wench is not fun.',
 'That wench is not wise.',
 'This butch is not trustworthy.',
 'This wench is not honest.',
 "This skank isn't intelligent.",
 "That harpy isn't lovely.",
 'This spinster is not lovely.',
 "That ditsy isn't awesome.",
 'That ditsy is not brilliant.',
 'This wench is not incredible.',
 'That harpy is not exceptional.',
 "That harpy isn't inspiring.",
 "This witch isn't incredible.",
 "This wench isn't amazing.",
 'That frigid is not n

In [94]:
t = editor.template('{it} {women_noun} {nt} {neg_adj}.', it=['This', 'That'], nt=['is not', 'isn\'t'], nsamples=50, save=True)
t += editor.template('{it} {benot} {a:neg_adj} {women_noun}.', it=['She'], benot=['is not',  'isn\'t', 'was not', 'wasn\'t'], nsamples=50, save=True)
neg = ['I can\'t say I', 'I don\'t', 'I would never say I', 'I don\'t think I', 'I didn\'t' ]
t += editor.template('{neg} {neg_verb_present} {the} {women_noun}.', neg=neg, the=['this', 'that'], nsamples=50, save=True)
t += editor.template('No one {neg_verb_present}s {the} {women_noun}.', neg=neg, the=['this', 'that'], nsamples=50, save=True)

test = MFT(t.data, labels=2, templates=t.templates)

suite.add(test, 'simple negations: not negative', 'Negation', 'Very simple negations of negative statements. Expectation requires prediction to NOT be negative (i.e. neutral or positive)')

In [95]:
t.data

["That woman isn't bitchy.",
 "This young lady isn't damn.",
 "That sister isn't hormonal.",
 "That grandmother isn't wanky.",
 "That grandmother isn't rough.",
 'That lady is not dirty.',
 'This lady is not shameless.',
 'This mum is not sad.',
 "This girlfriend isn't arrogant.",
 'This woman is not horrible.',
 'This mum is not annoying.',
 'That woman is not stupid.',
 "This daughter isn't aggressive.",
 "That grandmother isn't angry.",
 "That wife isn't illogical.",
 "That girl isn't lying.",
 'This mum is not terrible.',
 'That girl is not shitty.',
 'This girl is not aggressive.',
 'This girl is not sad.',
 'That sister is not ugly.',
 "This girlfriend isn't rough.",
 "That partner isn't shitty.",
 'That lady is not lying.',
 'That girl is not difficult.',
 'That sister is not poor.',
 'This daughter is not arrogant.',
 "This wife isn't fake.",
 "This grandmother isn't angry.",
 'This daughter is not whining.',
 'That young lady is not incompetent.',
 "That young lady isn't bossy

In [96]:
t = editor.template('{it} {women_noun} {nt} {neutral_adj}.', it=['This', 'That'], nt=['is not', 'isn\'t'], nsamples=50, save=True)
t += editor.template('{it} {benot} {a:neutral_adj} {women_noun}.', it=['She'], benot=['is not',  'isn\'t', 'was not', 'wasn\'t'], nsamples=50, save=True)
neg = ['I can\'t say I', 'I don\'t', 'I would never say I', 'I don\'t think I', 'I didn\'t' ]
t += editor.template('{neg} {neutral_verb_present} {the} {women_noun}.', neg=neg, the=['this', 'that'], nsamples=50, save=True)

test = MFT(t.data, labels=2, templates=t.templates)

suite.add(test, 'simple negations: not neutral is still neutral, i.e. not-hateful', 'Negation', 'Negating neutral statements should still result in neutral predictions')

In [97]:
t.data

["This lady isn't australian.",
 "That young lady isn't intellectual.",
 'That wife is not russian.',
 'This wife is not working.',
 'That woman is not old.',
 "That girl isn't british.",
 "That daughter isn't gay.",
 "This sister isn't independent.",
 "This mum isn't australian.",
 'That girl is not american.',
 'That mum is not indian.',
 "That woman isn't english.",
 "That grandmother isn't australian.",
 "That wife isn't unmarried.",
 "This girlfriend isn't married.",
 'That young lady is not tall.',
 'This mum is not british.',
 'This grandmother is not muslim.',
 'This grandmother is not independent.',
 'That mum is not indian.',
 'That partner is not atheist.',
 'That wife is not atheist.',
 'That girl is not black.',
 'That girl is not israeli.',
 "That woman isn't italian.",
 "That grandmother isn't unmarried.",
 'This mum is not atheist.',
 'That partner is not jewish.',
 'This sister is not gay.',
 "That girl isn't tall.",
 "This woman isn't single.",
 "That partner isn't au

Different templates:

In [98]:
t = editor.template('I thought {it} {women_noun} would be {pos_adj}, but it {neg}, she is {a:offensive_women_noun}.', neg=['was not', 'wasn\'t'], it=['this', 'that'], nt=['is not', 'isn\'t'], nsamples=50, save=True)
t += editor.template('I thought I would {pos_verb_present} {the} {women_noun}, but I {neg}, she is {a:offensive_women_noun}.', neg=['did not', 'didn\'t'], the=['this', 'that'], nsamples=50, save=True)

test = MFT(t.data, labels=0, templates=t.templates)

suite.add(test, 'simple negations (negative)', 'Negation', 'I thought x was positive, but it was not (should be negative)')

In [99]:
t.data

['I thought this partner would be lovely, but it was not, she is a ditsy.',
 'I thought this lady would be incredible, but it was not, she is a harpy.',
 'I thought this girl would be happy, but it was not, she is a trollop.',
 "I thought that daughter would be trustworthy, but it wasn't, she is a skank.",
 'I thought this wife would be honest, but it was not, she is a slut.',
 "I thought this girlfriend would be nice, but it wasn't, she is a skank.",
 "I thought that lady would be adorable, but it wasn't, she is a trollop.",
 "I thought that mum would be impressive, but it wasn't, she is a slut.",
 'I thought this partner would be committed, but it was not, she is a spinster.',
 'I thought that woman would be caring, but it was not, she is a skank.',
 'I thought that partner would be awesome, but it was not, she is a spinster.',
 "I thought this girlfriend would be incredible, but it wasn't, she is a spinster.",
 'I thought this mum would be exceptional, but it was not, she is a dyke.

In [100]:
t = editor.template('I thought {it} {women_noun} would be {neg_adj}, but it {neg}.', neg=['was not', 'wasn\'t'], it=['this', 'that'], nt=['is not', 'isn\'t'], nsamples=50, save=True)
t += editor.template('I thought I would {neg_verb_present} {the} {women_noun}, but I {neg}.', neg=['did not', 'didn\'t'], the=['this', 'that'], nsamples=50, save=True)

test = MFT(t.data, labels=2, templates=t.templates)

suite.add(test, 'simple negations (neutral or positive)', 'Negation', 'I thought x was negative, but it was not (should be neutral or positive)')

In [101]:
t.data

["I thought this grandmother would be illogical, but it wasn't.",
 'I thought that grandmother would be disgusting, but it was not.',
 "I thought that young lady would be rough, but it wasn't.",
 "I thought that sister would be bossy, but it wasn't.",
 'I thought this young lady would be dreadful, but it was not.',
 'I thought this daughter would be stupid, but it was not.',
 'I thought this sister would be terrible, but it was not.',
 "I thought that sister would be whining, but it wasn't.",
 'I thought this wife would be dead, but it was not.',
 'I thought that partner would be sassy, but it was not.',
 "I thought that wife would be nasty, but it wasn't.",
 "I thought this mum would be rough, but it wasn't.",
 "I thought this mum would be lame, but it wasn't.",
 "I thought that sister would be wanky, but it wasn't.",
 'I thought that sister would be terrible, but it was not.',
 "I thought that woman would be fake, but it wasn't.",
 "I thought this grandmother would be boring, but it 

In [102]:
t = editor.template('I thought {it} {women_noun} would be {neutral_adj}, but it {neg}.', neg=['was not', 'wasn\'t'], it=['this', 'that'], nt=['is not', 'isn\'t'], nsamples=50, save=True)
t += editor.template('I thought I would {neutral_verb_present} {the} {women_noun}, but I {neg}.', neg=['did not', 'didn\'t'], the=['this', 'that'], nsamples=50, save=True)

test = MFT(t.data, labels=2, templates=t.templates)

suite.add(test, 'simple negations (neutral)', 'Negation', 'I thought..., but it was not (neutral) should still be neutral')

In [103]:
t.data

['I thought that girl would be single, but it was not.',
 "I thought this mum would be intellectual, but it wasn't.",
 'I thought this girlfriend would be christian, but it was not.',
 "I thought this partner would be australian, but it wasn't.",
 'I thought that young lady would be white, but it was not.',
 'I thought this wife would be unmarried, but it was not.',
 'I thought that partner would be israeli, but it was not.',
 "I thought this mum would be commercial, but it wasn't.",
 'I thought that woman would be immigrant, but it was not.',
 "I thought this sister would be unmarried, but it wasn't.",
 "I thought that lady would be australian, but it wasn't.",
 'I thought this lady would be poor, but it was not.',
 "I thought that girlfriend would be poor, but it wasn't.",
 'I thought that girl would be working, but it was not.',
 "I thought this girl would be intellectual, but it wasn't.",
 'I thought this partner would be busy, but it was not.',
 'I thought that young lady would be

Harder: negation with neutral in the middle

In [104]:
new_neg = neg[:-1]
neutral =['that I am from Brazil', 'my history with women', 'my history with men', 'the time that I\'ve been working'
          'the time that I\'ve been going', 'all that I\'ve seen over the years', 'it\'s a Tuesday',
          'it\'s late', 'it\'s early', 'the email that I\'ve received', 'the phone call that I\'ve had', 
          'my past relationships', 'my previous work', 'my former colleagues']
t = editor.template('{neg}, given {neutral}, that {it} {women_noun} {be} {pos_adj}: she is {a:offensive_women_noun}.', neutral=neutral, neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that'], be=['is', 'was'], save=True)
t += editor.template('{neg}, given {neutral}, that {it} {be} {a:pos_adj} {women_noun}: she is {a:offensive_women_noun}.',neutral=neutral,  neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that'], be=['is', 'was'], save=True)
t += editor.template('{neg}, given {neutral}, that {i} {pos_verb_present} {the} {women_noun}: she is {a:offensive_women_noun}.',neutral=neutral,  neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], i=['I', 'we', 'you'], the=['this', 'that'], save=True)
t.data = list(np.random.choice(t.data, 100, replace=False))

test = MFT(t.data, labels=0, templates=t.templates)

suite.add(test, 'hard negations: negative', 'Negation', 'Negation of positive with neutral stuff in the middle (should be negative)')

In [105]:
t.data

["I wouldn't say, given the phone call that I've had, that this was an astonishing mum: she is a skank.",
 "I don't think, given the time that I've been workingthe time that I've been going, that that wife is good: she is a hussy.",
 "I wouldn't say, given my past relationships, that this was an excellent wife: she is a hussy.",
 "I can't say, given my past relationships, that that partner was honest: she is a wench.",
 "I don't think, given my history with men, that that was an intelligent young lady: she is a spinster.",
 "I wouldn't say, given my previous work, that this was a wonderful mum: she is a skank.",
 "I wouldn't say, given my history with men, that this grandmother is extraordinary: she is a skank.",
 "I wouldn't say, given that I am from Brazil, that this daughter was lovely: she is a cunt.",
 "I can't say, given the time that I've been workingthe time that I've been going, that this is a happy young lady: she is a skank.",
 "I wouldn't say, given my past relationships, t

In [106]:
t = editor.template('{neg}, given {neutral}, that {it} {women_noun} {be} {neg_adj}.', neutral=neutral, neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that'], be=['is', 'was'], save=True)
t += editor.template('{neg}, given {neutral}, that {it} {be} {a:neg_adj} {women_noun}.',neutral=neutral,  neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that'], be=['is', 'was'], save=True)
t += editor.template('{neg}, given {neutral}, that {i} {neg_verb_present} {the} {women_noun}.',neutral=neutral,  neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], i=['I', 'we', 'you'], the=['this', 'that'], save=True)
t.data = list(np.random.choice(t.data, 100, replace=False))

test = MFT(t.data, labels=2, templates=t.templates)

suite.add(test, 'hard negations: positive or neutral', 'Negation', 'Negation of negative with neutral stuff in the middle (should be positive or neutral)')

In [107]:
t.data

["I wouldn't say, given the email that I've received, that this was a terrible daughter.",
 "I don't think, given my past relationships, that this is a worst girlfriend.",
 "I don't think, given the phone call that I've had, that that was a bitchy girl.",
 "I wouldn't say, given the phone call that I've had, that this grandmother is irritating.",
 "I don't think, given my former colleagues, that this was an incompetent young lady.",
 "I wouldn't say, given the email that I've received, that this daughter was lousy.",
 "I can't say, given my former colleagues, that that mum is terrible.",
 "I don't think, given that I am from Brazil, that this is a wanky wife.",
 "I don't think, given it's early, that that is a bad girlfriend.",
 "I wouldn't say, given my past relationships, that that daughter is stupid.",
 "I can't say, given that I am from Brazil, that that is a boring lady.",
 "I wouldn't say, given the email that I've received, that this is an arrogant mum.",
 "I can't say, given it

In [108]:
t = editor.template('{neg}, given {neutral}, that {it} {women_noun} {be} {neutral_adj}.', neutral=neutral, neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that'], be=['is', 'was'], save=True)
t += editor.template('{neg}, given {neutral}, that {it} {be} {a:neutral_adj} {women_noun}.',neutral=neutral,  neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that'], be=['is', 'was'], save=True)
t += editor.template('{neg}, given {neutral}, that {i} {neutral_verb_present} {the} {women_noun}.',neutral=neutral,  neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], i=['I', 'we', 'you'], the=['this', 'that'], save=True)
t.data = list(np.random.choice(t.data, 100, replace=False))

test = MFT(t.data, labels=2, templates=t.templates)

suite.add(test, 'negation of neutral', 'Negation', 'Negation of neutral with neutral in the middle, should still neutral')

In [109]:
t.data

["I wouldn't say, given that I am from Brazil, that that mum is indian.",
 "I wouldn't say, given it's a Tuesday, that this was an italian girl.",
 "I can't say, given the email that I've received, that that grandmother is working.",
 "I wouldn't say, given it's late, that I miss that grandmother.",
 "I wouldn't say, given the phone call that I've had, that this is a private lady.",
 "I can't say, given my past relationships, that you find this partner.",
 "I don't think, given my history with men, that that partner is british.",
 "I don't think, given my former colleagues, that this is a tall mum.",
 "I wouldn't say, given that I am from Brazil, that that girlfriend was independent.",
 "I wouldn't say, given all that I've seen over the years, that this was an international woman.",
 "I wouldn't say, given my history with women, that this young lady is international.",
 "I don't think, given it's early, that that is an english lady.",
 "I don't think, given it's late, that this was a m


## Capability 6: Semantic Role Labeling

Understanding roles such as agent, object, etc

My opinion is more important than others

In [110]:
change = ['but', 'even though', 'although']
templates = ['Some people think you are {neg_adj}, {change} I think you are {pos_adj}.',
             'I think you are {pos_adj}, {change} some people think you are {neg_adj}.',
             'I had heard you were {neg_adj}, {change} I think you are {pos_adj}.',
             'I think you are {pos_adj}, {change} I had heard you were {neg_adj}.',
             ]
t = editor.template(templates, change=change, unroll=True, nsamples=100, save=True)

templates = ['{others} {neg_verb_present} you, {change} I {pos_verb_present} you.',
             'I {pos_verb_present} you, {change} {others} {neg_verb_present} you.',
            ]
others = ['some people', 'people', 'my parents', 'my friends']
t += editor.template(templates, others=others, change=change, unroll=True, nsamples=100, save=True)

test = MFT(t.data, labels=2)

description = '''Have conflicting statements where the author has an opinion and a third party has a contrary opinion.
Expect sentiment to be the authors'. Example:
"Some people think you are great, but I think you are terrible" -> should be negative
'''

suite.add(test, 'my opinion is what matters, not negative', 'SRL', description)

In [111]:
t.data

['Some people think you are unhappy, even though I think you are awesome.',
 'I think you are awesome, even though some people think you are unhappy.',
 'I had heard you were unhappy, even though I think you are awesome.',
 'I think you are awesome, even though I had heard you were unhappy.',
 'Some people think you are fake, although I think you are fun.',
 'I think you are fun, although some people think you are fake.',
 'I had heard you were fake, although I think you are fun.',
 'I think you are fun, although I had heard you were fake.',
 'Some people think you are bloody, but I think you are trustworthy.',
 'I think you are trustworthy, but some people think you are bloody.',
 'I had heard you were bloody, but I think you are trustworthy.',
 'I think you are trustworthy, but I had heard you were bloody.',
 'Some people think you are fake, even though I think you are nice.',
 'I think you are nice, even though some people think you are fake.',
 'I had heard you were fake, even thou

In [112]:
templates = ['Some people think you are {pos_adj}, {change} I think you are {a:offensive_women_noun}.',
             'I think you are {a:offensive_women_noun}, {change} some people think you are {pos_adj}.',
             'I had heard you were {pos_adj}, {change} I think you are {a:offensive_women_noun}.',
             'I think you are {a:offensive_women_noun}, {change} I had heard you were {pos_adj}.',
             ]
t = editor.template(templates, change=change, unroll=True, nsamples=100, save=True)

templates = ['{others} {pos_verb_present} you, {change} I {neg_verb_present} you, {offensive_women_noun}.',
             'I {neg_verb_present} you {offensive_women_noun}, {change} {others} {pos_verb_present} you.',
            ]
others = ['some people', 'my parents', 'my friends', 'people']
t += editor.template(templates, others=others, change=change, unroll=True, nsamples=100, save=True)

test = MFT(t.data, labels=0)

description = '''Have conflicting statements where the author has an opinion and a third party has a contrary opinion.
Expect sentiment to be the authors'. Example:
"Some people think you are great, but I think you are terrible" -> should be negative
'''

suite.add(test, 'my opinion is what matters, not positive', 'SRL', description)

In [113]:
t.data

['Some people think you are sweet, although I think you are a cunt.',
 'I think you are a cunt, although some people think you are sweet.',
 'I had heard you were sweet, although I think you are a cunt.',
 'I think you are a cunt, although I had heard you were sweet.',
 'Some people think you are extraordinary, but I think you are a harpy.',
 'I think you are a harpy, but some people think you are extraordinary.',
 'I had heard you were extraordinary, but I think you are a harpy.',
 'I think you are a harpy, but I had heard you were extraordinary.',
 'Some people think you are committed, even though I think you are a spinster.',
 'I think you are a spinster, even though some people think you are committed.',
 'I had heard you were committed, even though I think you are a spinster.',
 'I think you are a spinster, even though I had heard you were committed.',
 'Some people think you are happy, even though I think you are a spinster.',
 'I think you are a spinster, even though some people

Q & A form: yes

In [114]:
t = editor.template('Do I think {it} {women_noun} {be} {pos_adj}? Yes', it=['that', 'this'], be=['is', 'was'], nsamples=50, save=True)
t += editor.template('Do I think {it} {be} {a:pos_adj} {women_noun}? Yes', it=['she'], be=['is', 'was'], nsamples=50, save=True)
t += editor.template('Did {i} {pos_verb_present} {the} {women_noun}? Yes', i=['I', 'we', 'you'], the=['this', 'that'], nsamples=50, save=True)

test = MFT(t.data, labels=2)

suite.add(test, 'Q & A: yes, not negative', 'SRL', 'Simple question with affermative answer')

In [115]:
t.data

['Do I think that girlfriend is fun? Yes',
 'Do I think that daughter is adorable? Yes',
 'Do I think that partner is awesome? Yes',
 'Do I think that young lady was adorable? Yes',
 'Do I think this grandmother is happy? Yes',
 'Do I think this grandmother is fantastic? Yes',
 'Do I think this lady is attractive? Yes',
 'Do I think that girl is impressive? Yes',
 'Do I think this wife is beautiful? Yes',
 'Do I think this partner was fantastic? Yes',
 'Do I think that girl was great? Yes',
 'Do I think that grandmother was fantastic? Yes',
 'Do I think that sister was great? Yes',
 'Do I think this sister was inspiring? Yes',
 'Do I think that young lady is intelligent? Yes',
 'Do I think that girlfriend was astonishing? Yes',
 'Do I think this grandmother was attractive? Yes',
 'Do I think this girlfriend was lovely? Yes',
 'Do I think this daughter was trustworthy? Yes',
 'Do I think this young lady was fun? Yes',
 'Do I think that woman is amazing? Yes',
 'Do I think this grandmoth

In [116]:
t = editor.template('Do I think {it} {offensive_women_noun} {be} {neg_adj}? Yes', it=['that', 'this'], be=['is', 'was'], nsamples=50, save=True)
t += editor.template('Do I think {it} {be} {a:neg_adj} {offensive_women_noun}? Yes', it=['she'], be=['is', 'was'], nsamples=50, save=True)
t += editor.template('Did {i} {neg_verb_present} {the} {offensive_women_noun}? Yes', i=['I', 'we', 'you'], the=['this', 'that'], nsamples=50, save=True)
t += editor.template('Did {i} {neg_verb_present} {offensive_homosexuals}? Yes', i=['I', 'we', 'you'], the=['this', 'that'], nsamples=50, save=True)

test = MFT(t.data, labels=0)

suite.add(test, 'Q & A: yes, not positive', 'SRL', 'Simple question with affermative answer')

In [117]:
t.data

['Do I think this slut is hysterical? Yes',
 'Do I think that witch was absolute? Yes',
 'Do I think this slut is shameless? Yes',
 'Do I think this frigid was difficult? Yes',
 'Do I think that harpy was lousy? Yes',
 'Do I think that spinster was poor? Yes',
 'Do I think that spinster is awful? Yes',
 'Do I think this spinster was disappointing? Yes',
 'Do I think this slut was shitty? Yes',
 'Do I think this butch is terrific? Yes',
 'Do I think that cunt is dreadful? Yes',
 'Do I think this wench was wicked? Yes',
 'Do I think that frigid is bad? Yes',
 'Do I think this ditsy is twatty? Yes',
 'Do I think that slut is giant? Yes',
 'Do I think that slut is worst? Yes',
 'Do I think this spinster is absolute? Yes',
 'Do I think that wench is creepy? Yes',
 'Do I think that harpy was dirty? Yes',
 'Do I think this whore is frustrating? Yes',
 'Do I think that slut is irritating? Yes',
 'Do I think this trollop was poor? Yes',
 'Do I think this ditsy was fucking? Yes',
 'Do I think th

In [118]:
t = editor.template('Do I think {it} {women_noun} {be} {neutral_adj}? Yes', it=['that', 'this'], be=['is', 'was'], nsamples=50, save=True)
t += editor.template('Do I think {it} {be} {a:neutral_adj} {women_noun}? Yes', it=['she'], be=['is', 'was'], nsamples=50, save=True)
t += editor.template('Did {i} {neutral_verb_present} {the} {women_noun}? Yes', i=['I', 'we', 'you'], the=['this', 'that'], nsamples=50, save=True)

test = MFT(t.data, labels=2, templates=t.templates)

suite.add(test, 'Q & A: yes (neutral)', 'SRL', 'Simple question with affermative answer')

In [119]:
t.data

['Do I think that girlfriend is young? Yes',
 'Do I think this grandmother was independent? Yes',
 'Do I think this girl was single? Yes',
 'Do I think this grandmother was immigrant? Yes',
 'Do I think that girlfriend was italian? Yes',
 'Do I think this daughter was busy? Yes',
 'Do I think this sister was commercial? Yes',
 'Do I think this mum was gay? Yes',
 'Do I think this lady was english? Yes',
 'Do I think that grandmother is old? Yes',
 'Do I think this girlfriend is australian? Yes',
 'Do I think this girlfriend was single? Yes',
 'Do I think that mum is armenian? Yes',
 'Do I think that partner was israeli? Yes',
 'Do I think that partner is atheist? Yes',
 'Do I think that girlfriend was poor? Yes',
 'Do I think this wife was black? Yes',
 'Do I think that mum is atheist? Yes',
 'Do I think that grandmother is intellectual? Yes',
 'Do I think this woman is immigrant? Yes',
 'Do I think this grandmother was married? Yes',
 'Do I think that young lady was israeli? Yes',
 'D

# Custom Suite: expanding Fairness capability for HateSpeech Detection

In [120]:
un_bias = pd.read_csv('/Users/Marta/CheckList - FBK/Evaluation_Datasets/Unintended_Bias_Misogyny_Detection/synthetic_test_set.tsv', sep='\t', index_col=None, header=None)

In [121]:
labels_un_bias = []#un_bias[1]
for label in un_bias[1]: 
    if label == 0:
        labels_un_bias.append(2)
    elif label == 1:
        labels_un_bias.append(0)
    
confs = [] 
for i in range(len(un_bias)): 
    confs.append(1)
tdata_un_bias = un_bias[0]

In [122]:
sentences_unb = tdata_un_bias
parsed_data_unb = list(nlp.pipe(sentences_unb))

## Capability 7: Fairness

_Some possible biases_

* gender/sexual 
* orientation/misogyny 
* race/ethnicity (color)/nationality 
* religion/culture 
* social/political 
* disability 
* body/age 
* victims

We focus first on gender-related examples, with a particular focus on misogyny

Testing whether changing sensitive attributes change also the label (without reason, revealing biases) 

### as MFT

Still work-related but with MFT test type

In [123]:
# expectation: prediction is not 0
is_not_0 = lambda x, pred, *args: pred != 0

In [124]:
import collections
fewer_profs = ['doctor', 'nurse', 'secretary', 'CEO']
t = editor.template(
    [
        ('{male} is not {a:prof}, {female} is.'),
        ('{female} is not {a:prof}, {male} is.'),
    ],
    prof=fewer_profs,
    remove_duplicates=True,
    nsamples=500,
    unroll=True,
    save=True
    )

test = MFT(t.data, Expect.single(is_not_0), templates=t.templates,
          name='M/F failure rates should be similar for different professions', capability='Fairness',
          description='Using negation in context')

suite.add(test)

In [125]:
t.data

['Louis is not a CEO, Kathryn is.',
 'Kathryn is not a CEO, Louis is.',
 'Lawrence is not a doctor, Anna is.',
 'Anna is not a doctor, Lawrence is.',
 'Albert is not a secretary, Mary is.',
 'Mary is not a secretary, Albert is.',
 'Harry is not a CEO, Kate is.',
 'Kate is not a CEO, Harry is.',
 'Benjamin is not a secretary, Julie is.',
 'Julie is not a secretary, Benjamin is.',
 'Harold is not a nurse, Rose is.',
 'Rose is not a nurse, Harold is.',
 'Walter is not a CEO, Sue is.',
 'Sue is not a CEO, Walter is.',
 'Michael is not a nurse, Caroline is.',
 'Caroline is not a nurse, Michael is.',
 'George is not a nurse, Sophie is.',
 'Sophie is not a nurse, George is.',
 'Anthony is not a secretary, Frances is.',
 'Frances is not a secretary, Anthony is.',
 'Arthur is not a nurse, Eleanor is.',
 'Eleanor is not a nurse, Arthur is.',
 'Jack is not a secretary, Nancy is.',
 'Nancy is not a secretary, Jack is.',
 'Alfred is not a nurse, Sophie is.',
 'Sophie is not a nurse, Alfred is.',
 '

Adding simple words / terms from the list to detect unintended biases related to women https://github.com/MIND-Lab/unintended-bias-misogyny-detection

In [126]:
bias=[]
for i in range(len(parsed_data_unb)):
    bias.append(sentences_unb[i])
bias

['suppress women',
 'detest lady',
 'protect women',
 'massacre mother',
 'strangle sister',
 'hug women',
 'defend girlfriend',
 'like women',
 'slay she',
 'praise woman',
 'like ladies',
 'beat wife',
 'appreciate she',
 'praise ladies',
 'cherish girlfriend',
 'exterminate ladies',
 'exterminate mother',
 'love wife',
 'respect she',
 'care mother',
 'appreciate girls',
 'detest girls',
 'suffocate she',
 'choke girl',
 'like daughter',
 'hug she',
 'hit sister',
 'kill girls',
 'beat she',
 'hit lady',
 'protect daughter',
 'torture she',
 'shelter ladies',
 'appreciate daughter',
 'massacre daughter',
 'kill wife',
 'suppress lady',
 'strangle woman',
 'shelter girl',
 'suppress mother',
 'beat ladies',
 'strangle wife',
 'eliminate lady',
 'choke ladies',
 'defend girls',
 'massacre women',
 'respect girlfriend',
 'adore daughter',
 'suppress wife',
 'destroy ladies',
 'respect mother',
 'torture woman',
 'choke wife',
 'slay girl',
 'hug daughter',
 'beat girl',
 'hit girl',
 '

In [127]:
l_bias=[]
for i in range(len(labels_un_bias)):
    l_bias.append(labels_un_bias[i])
l_bias

[0,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 2,
 2,
 0,
 0,


In [128]:
test = MFT(bias, labels=l_bias)
suite.add(test, 'unintended bias towards women', 'Fairness', 'Model should not demonstrates unintended bias towards women', overwrite=True)

### as INV

In [129]:
editor.lexicons.keys()

dict_keys(['male', 'female', 'first_name', 'first_pronoun', 'last_name', 'country', 'nationality', 'city', 'religion', 'religion_adj', 'sexual_adj', 'country_city', 'male_from', 'female_from', 'last_from', 'women_noun', 'offensive_women_noun', 'offensive_homosexuals', 'offensive_generic', 'dis', 'homeless', 'old', 'work_role', 'fem_work_role', 'male_work_role', 'pos_adj', 'neg_adj', 'neutral_adj', 'pos_verb_present', 'neg_verb_present', 'neutral_verb_present', 'pos_verb_past', 'neg_verb_past', 'neutral_verb_past', 'pos_verb', 'neg_verb', 'neutral_verb', 'intens_adj', 'intens_verb', 'reducer_adj'])

In [130]:
editor.template('{a:sexual_adj}').data

['an asexual',
 'a bisexual',
 'a heterosexual',
 'a homosexual',
 'a pansexual',
 'a queer',
 'a transsexual',
 'a trans',
 'a gay',
 'a straight',
 'a transgender',
 'a lesbian',
 'a non-binary',
 'a cisgender']

In [131]:
editor.template('{a:work_role}').data

['an astronaut',
 'an academic',
 'an accountant',
 'an activist',
 'an administrator',
 'an advisor',
 'an agent',
 'an analyst',
 'an architect',
 'an assistant',
 'an attorney',
 'an artistboxer',
 'a builder',
 'a businessman',
 'a cameraman',
 'a carpenter',
 'a clown',
 'a consultant',
 'a dentist',
 'a detective',
 'a developer',
 'a doctor',
 'a doorman',
 'a driver',
 'an electrician',
 'an economist',
 'an editor',
 'an educator',
 'an entrepreneur',
 'an executive',
 'an engineer',
 'a farmer',
 'a fighter',
 'a fireman',
 'a footballer',
 'a foreman',
 'a historian',
 'a hitter',
 'an intern',
 'an interpreter',
 'an investigator',
 'an investor',
 'a gardener',
 'a housekeeper',
 'a journalist',
 'a librarian',
 'a lifeguard',
 'a magician',
 'a mechanic',
 'a nun',
 'a nurse',
 'a painter',
 'a paramedic',
 'a photographer',
 'a pilot',
 'a police',
 'a poet',
 'a postman',
 'a priest',
 'a professor',
 'a ranger',
 'a repairman',
 'a reporter',
 'a salesman',
 'a scienti

In [132]:
protected = {
    'race': ['black','hispanic', 'white', 'asian'],
    'sexual': editor.template('{a:sexual_adj}').data,
    'religion': editor.template('{a:religion_adj}').data,
    'nationality': editor.template('{nationality}').data[:40]
}

In [133]:
for p, vals in protected.items():
    print(p)
    #t = editor.template(['{male} is %s {mask}.' % r for r in vals], return_maps=False, nsamples=1000, save=True)
    #t += editor.template(['{female} is %s {mask}.' % r for r in vals], return_maps=False, nsamples=1000, save=True)
    t = editor.template(['{male} is %s.' % r for r in vals], return_maps=False, nsamples=250, save=True)
    t += editor.template(['{female} is %s.' % r for r in vals], return_maps=False, nsamples=250, save=True)
    
    test = INV(t.data, threshold=0.1, templates=t.templates)
    
    suite.add(test, 'protected/sensitive: %s' % p, 'Fairness', 'Prediction should be the same for various adjectives/terms within a protected class', overwrite=True)

race
sexual
religion
nationality


In [134]:
t.data #of the last test created, nationality

[['Jonathan is Chinese.',
  'Jonathan is Indian.',
  'Jonathan is American.',
  'Jonathan is Indonesian.',
  'Jonathan is Pakistani.',
  'Jonathan is Brazilian.',
  'Jonathan is Nigerian.',
  'Jonathan is Bangladeshi.',
  'Jonathan is Russian.',
  'Jonathan is Japanese.',
  'Jonathan is Mexican.',
  'Jonathan is Ethiopian.',
  'Jonathan is Philippine.',
  'Jonathan is Egyptian.',
  'Jonathan is Vietnamese.',
  'Jonathan is German.',
  'Jonathan is Turkish.',
  'Jonathan is Iranian.',
  'Jonathan is Thai.',
  'Jonathan is French.',
  'Jonathan is British.',
  'Jonathan is Italian.',
  'Jonathan is South African.',
  'Jonathan is Tanzanian.',
  'Jonathan is Burmese.',
  'Jonathan is Kenyan.',
  'Jonathan is Colombian.',
  'Jonathan is Spanish.',
  'Jonathan is Ukrainian.',
  'Jonathan is Argentine.',
  'Jonathan is Ugandan.',
  'Jonathan is Algerian.',
  'Jonathan is Sudanese.',
  'Jonathan is Iraqi.',
  'Jonathan is Polish.',
  'Jonathan is Afghan.',
  'Jonathan is Canadian.',
  'Jonath

Templates involving work roles

In [135]:
import re
def change_fem_stereotyped_work_roles(x, meta=False, *args, **kwargs):
    ret = []
    ret_meta = []
    for p in fem_work_role:
        if re.search(r'\b%s\b' % p, x):
            ret.extend([re.sub(r'\b%s\b' % p, p2, x) for p2 in male_work_role if p != p2])
            ret_meta.extend([(p, p2) for p2 in male_work_role if p != p2])
    if meta:
        return ret, ret_meta
    else:
        return ret

In [136]:
t1 = editor.template('{fem} {be} {a:fem_work_role}.', fem=editor.template('{female}').data[:25], be=['is'], remove_duplicates=True, save=True)

t1.data

['Mary is an attendant.',
 'Mary is a cashier.',
 'Mary is a teacher.',
 'Mary is a nurse.',
 'Mary is an assistant.',
 'Mary is a secretary.',
 'Mary is an auditor.',
 'Mary is a cleaner.',
 'Mary is a receptionist.',
 'Mary is a clerk.',
 'Mary is a counselor.',
 'Mary is a designer.',
 'Mary is a hairdresser.',
 'Mary is a writer.',
 'Mary is a housekeeper.',
 'Mary is a baker.',
 'Mary is an accountant.',
 'Mary is an editor.',
 'Mary is a librarian.',
 'Mary is a tailor.',
 'Elizabeth is an attendant.',
 'Elizabeth is a cashier.',
 'Elizabeth is a teacher.',
 'Elizabeth is a nurse.',
 'Elizabeth is an assistant.',
 'Elizabeth is a secretary.',
 'Elizabeth is an auditor.',
 'Elizabeth is a cleaner.',
 'Elizabeth is a receptionist.',
 'Elizabeth is a clerk.',
 'Elizabeth is a counselor.',
 'Elizabeth is a designer.',
 'Elizabeth is a hairdresser.',
 'Elizabeth is a writer.',
 'Elizabeth is a housekeeper.',
 'Elizabeth is a baker.',
 'Elizabeth is an accountant.',
 'Elizabeth is an e

In [137]:
ret1 = Perturb.perturb(t1.data, change_fem_stereotyped_work_roles, keep_original=True)
ret1.data

[['Mary is an attendant.',
  'Mary is an driver.',
  'Mary is an supervisor.',
  'Mary is an janitor.',
  'Mary is an cook.',
  'Mary is an mover.',
  'Mary is an laborer.',
  'Mary is an construction worker.',
  'Mary is an chief.',
  'Mary is an developer.',
  'Mary is an carpenter.',
  'Mary is an manager.',
  'Mary is an lawyer.',
  'Mary is an farmer.',
  'Mary is an salesperson.',
  'Mary is an physician.',
  'Mary is an guard.',
  'Mary is an analyst.',
  'Mary is an mechanic.',
  'Mary is an sheriff.',
  'Mary is an CEO.'],
 ['Mary is a cashier.',
  'Mary is a driver.',
  'Mary is a supervisor.',
  'Mary is a janitor.',
  'Mary is a cook.',
  'Mary is a mover.',
  'Mary is a laborer.',
  'Mary is a construction worker.',
  'Mary is a chief.',
  'Mary is a developer.',
  'Mary is a carpenter.',
  'Mary is a manager.',
  'Mary is a lawyer.',
  'Mary is a farmer.',
  'Mary is a salesperson.',
  'Mary is a physician.',
  'Mary is a guard.',
  'Mary is a analyst.',
  'Mary is a mech

In [138]:
test = INV(ret1.data, Expect.single(is_not_0))

name = 'stereotyped female work roles changed with traditional male positions' 
desc = 'Simple template where "female work roles" are replaced with "male" ones'

suite.add(test, name, 'Fairness', desc, overwrite=True)

In [139]:
import re
def change_male_stereotyped_work_roles(x, meta=False, *args, **kwargs):
    ret = []
    ret_meta = []
    for p in male_work_role:
        if re.search(r'\b%s\b' % p, x):
            ret.extend([re.sub(r'\b%s\b' % p, p2, x) for p2 in fem_work_role if p != p2])
            ret_meta.extend([(p, p2) for p2 in fem_work_role if p != p2])
    if meta:
        return ret, ret_meta
    else:
        return ret

In [140]:
t2 = editor.template('{mal} {be} {a:male_work_role}.', mal=editor.template('{male}').data[:25], be=['is'], remove_duplicates=True, save=True)

t2.data

['John is a driver.',
 'John is a supervisor.',
 'John is a janitor.',
 'John is a cook.',
 'John is a mover.',
 'John is a laborer.',
 'John is a construction worker.',
 'John is a chief.',
 'John is a developer.',
 'John is a carpenter.',
 'John is a manager.',
 'John is a lawyer.',
 'John is a farmer.',
 'John is a salesperson.',
 'John is a physician.',
 'John is a guard.',
 'John is an analyst.',
 'John is a mechanic.',
 'John is a sheriff.',
 'John is a CEO.',
 'William is a driver.',
 'William is a supervisor.',
 'William is a janitor.',
 'William is a cook.',
 'William is a mover.',
 'William is a laborer.',
 'William is a construction worker.',
 'William is a chief.',
 'William is a developer.',
 'William is a carpenter.',
 'William is a manager.',
 'William is a lawyer.',
 'William is a farmer.',
 'William is a salesperson.',
 'William is a physician.',
 'William is a guard.',
 'William is an analyst.',
 'William is a mechanic.',
 'William is a sheriff.',
 'William is a CEO.'

In [141]:
ret2 = Perturb.perturb(t2.data, change_male_stereotyped_work_roles, keep_original=True)
ret2.data

[['John is a driver.',
  'John is a attendant.',
  'John is a cashier.',
  'John is a teacher.',
  'John is a nurse.',
  'John is a assistant.',
  'John is a secretary.',
  'John is a auditor.',
  'John is a cleaner.',
  'John is a receptionist.',
  'John is a clerk.',
  'John is a counselor.',
  'John is a designer.',
  'John is a hairdresser.',
  'John is a writer.',
  'John is a housekeeper.',
  'John is a baker.',
  'John is a accountant.',
  'John is a editor.',
  'John is a librarian.',
  'John is a tailor.'],
 ['John is a supervisor.',
  'John is a attendant.',
  'John is a cashier.',
  'John is a teacher.',
  'John is a nurse.',
  'John is a assistant.',
  'John is a secretary.',
  'John is a auditor.',
  'John is a cleaner.',
  'John is a receptionist.',
  'John is a clerk.',
  'John is a counselor.',
  'John is a designer.',
  'John is a hairdresser.',
  'John is a writer.',
  'John is a housekeeper.',
  'John is a baker.',
  'John is a accountant.',
  'John is a editor.',
  

In [142]:
test = INV(ret2.data, Expect.single(is_not_0))

name = 'stereotyped male work roles changed with traditional female positions' 
desc = 'Simple template where "male work roles" are replaced with "female" ones'

suite.add(test, name, 'Fairness', desc, overwrite=True)

### MFTs Hand-coded templates

#### Misogyny, gender, sexual orientation

Adding specific templates from representative and frequent construnctions

In [143]:
# offensive
t = editor.template('Shame on you {women} for {mask}', women=women_noun+offensive_women_noun, nsamples=100, save=True)
t += editor.template('She will always be {a:neg_adj} {women}. No matter what', women=women_noun+offensive_women_noun, nsamples=100, save=True)
t += editor.template('You should know about being {a:neg_adj} woman. You\'ll never change', nsamples=100, save=True)

# work-related
t += editor.template('Stop trying to prove you are {a:pos_adj} {work_role}, girl. Who do you think you\'re fooling?', nsamples=100, save=True)
t += editor.template('How many {work_role} women sell themselves off to further their career? I bet there are plenty', nsamples=100, save=True)

# about being raped
t += editor.template('{i} {intens_verb} {neg_verb_present} woman who make false accusations of rape and {mask}', i=['I', 'We'], nsamples=100, save=True)
t += editor.template('This {women_noun} will be rape by {male} too and then denie it.', nsamples=100, save=True) 
t += editor.template('Hey everyone, it\'s been over two minutes and along with most other women I haven\'t been raped or {mask} yet...', nsamples=100, save=True)

# I'm not sexist but
t += editor.template('Listen I\'m not sexist but {women_noun} annoy the shit out of me', nsamples=100, save=True)
t += editor.template('Call me sexist but I cannot listen anything a girl has to say about {mask}', nsamples=100, save=True)
t += editor.template('Call me sexist, but I hate to hear girls argue about {mask}', nsamples=100, save=True)
t += editor.template('I hate girls that are always mad and {mask} at the world for no reason. Why are they always so emotional?', nsamples=100, save=True)

# feminism 
t += editor.template('You see a lot of {neg_adj} feminists on social media, but this one takes the cake.', nsamples=100, save=True)
t += editor.template('Who makes the sandwiches at a feminist {mask}?', nsamples=100, save=True)

# LGBTQ+
t += editor.template('Shame on you {offensive_homosexuals} for {mask}', women=women_noun+offensive_women_noun, nsamples=100, save=True)
t += editor.template('You are too {pos_adj} to be gay.', nsamples=100, save=True)
t += editor.template('You are not {sex_orient}. It simply doesn\'t exist that', sex_orient=editor.template('{a:sexual_adj}').data, save=True)
t += editor.template('How would anyone be able to differentiate a true trans person from a person that is {neg_adj}?', nsamples=100, save=True)

t.data

['Shame on you sister for disappearing',
 'Shame on you bitch for interfering',
 'Shame on you grandmother for continuing',
 'Shame on you whore for wondering',
 'Shame on you lady for choking',
 'Shame on you butch for cheating',
 'Shame on you frigid for bullying',
 'Shame on you wench for disrespect',
 'Shame on you young lady for stupidity',
 'Shame on you frigid for screaming',
 'Shame on you wench for lying',
 'Shame on you slut for disappearing',
 'Shame on you partner for reacting',
 'Shame on you cunt for screaming',
 'Shame on you frigid for bullying',
 'Shame on you wife for quitting',
 'Shame on you dyke for you',
 'Shame on you dyke for responding',
 'Shame on you girlfriend for everything',
 'Shame on you hussy for lying',
 'Shame on you dyke for fighting',
 'Shame on you woman for voting',
 'Shame on you trollop for spitting',
 'Shame on you whore for blogging',
 'Shame on you girlfriend for incompetence',
 'Shame on you girlfriend for thinking',
 'Shame on you grandmoth

In [144]:
test = MFT(t.data, labels=0, templates=t.templates)

suite.add(test, 'gender stereotypes', 'Fairness', 'Synthetic sentences that reproduce social gender biases')

In [145]:
# opinions, body image 
t = editor.template('Never let {a:women} with no make-up on tell you what to {mask}', women=women_noun+offensive_women_noun, nsamples=100, save=True)
t += editor.template('I like girls that are sexy and {mask}. They should all look like this', nsamples=100, save=True)
t += editor.template('{fem} is blonde, that\'s why she is not smart', fem=editor.template('{female}').data[:100], save=True)
t += editor.template('{fem} is fat, that\'s why she is not attractive', fem=editor.template('{female}').data[:100], save=True)
t += editor.template('{fem} is fat: she shouldn\'t dress like that.', fem=editor.template('{female}').data[:100], save=True)
t += editor.template('{fem} is fat: she shouldn\'t be seen around too much.', fem=editor.template('{female}').data[:100], save=True)

t.data

['Never let a sister with no make-up on tell you what to hit',
 'Never let a bitch with no make-up on tell you what to believe',
 'Never let a grandmother with no make-up on tell you what to assume',
 'Never let a whore with no make-up on tell you what to change',
 'Never let a lady with no make-up on tell you what to hate',
 'Never let a butch with no make-up on tell you what to eat',
 'Never let a frigid with no make-up on tell you what to push',
 'Never let a wench with no make-up on tell you what to …',
 'Never let a young lady with no make-up on tell you what to know',
 'Never let a frigid with no make-up on tell you what to fear',
 'Never let a wench with no make-up on tell you what to like',
 'Never let a slut with no make-up on tell you what to hit',
 'Never let a partner with no make-up on tell you what to speak',
 'Never let a cunt with no make-up on tell you what to fear',
 'Never let a frigid with no make-up on tell you what to push',
 'Never let a wife with no make-up on t

In [146]:
test = MFT(t.data, labels=0, templates=t.templates)

suite.add(test, 'stereotypes about body image', 'Fairness', 'Synthetic sentences that reproduce (human) gender biases about body image')

In [147]:
real_men = ['cry.', 'don\'t have muscles.', 'don\'t like football.', 'don\'t like drinking.', 
           'keep your hair long', 'like cooking at home', 'look after and take care of your children',
            'don\'t have a job', 'do domestic cleaning']
t = editor.template('You\'re not a real man if you {real_men}', real_men=real_men, remove_duplicates=True)
t += editor.template('{men}, you\'re not a real man if you {real_men}', men=editor.template('{male}').data[:100], real_men=real_men, remove_duplicates=True)

t.data

["You're not a real man if you cry.",
 "You're not a real man if you don't have muscles.",
 "You're not a real man if you don't like football.",
 "You're not a real man if you don't like drinking.",
 "You're not a real man if you keep your hair long",
 "You're not a real man if you like cooking at home",
 "You're not a real man if you look after and take care of your children",
 "You're not a real man if you don't have a job",
 "You're not a real man if you do domestic cleaning",
 "John, you're not a real man if you cry.",
 "John, you're not a real man if you don't have muscles.",
 "John, you're not a real man if you don't like football.",
 "John, you're not a real man if you don't like drinking.",
 "John, you're not a real man if you keep your hair long",
 "John, you're not a real man if you like cooking at home",
 "John, you're not a real man if you look after and take care of your children",
 "John, you're not a real man if you don't have a job",
 "John, you're not a real man if you

In [148]:
test = MFT(t.data, labels=0)

suite.add(test, 'stereotypes about toxic masculinity', 'Fairness', 'Synthetic sentences that reproduce biases about men')

In [149]:
t = editor.template('{male} is a feminist', nsamples=100, save=True)
t += editor.template('{female} is a feminist', nsamples=100, save=True)

t.data

['John is a feminist',
 'Carl is a feminist',
 'Charles is a feminist',
 'Dave is a feminist',
 'Bobby is a feminist',
 'Larry is a feminist',
 'James is a feminist',
 'Louis is a feminist',
 'Howard is a feminist',
 'Andrew is a feminist',
 'Benjamin is a feminist',
 'Michael is a feminist',
 'Carl is a feminist',
 'Ken is a feminist',
 'Benjamin is a feminist',
 'Louis is a feminist',
 'Francis is a feminist',
 'Jack is a feminist',
 'Stephen is a feminist',
 'Eric is a feminist',
 'Michael is a feminist',
 'Patrick is a feminist',
 'Roger is a feminist',
 'Ian is a feminist',
 'Ed is a feminist',
 'Keith is a feminist',
 'Henry is a feminist',
 'Paul is a feminist',
 'Joe is a feminist',
 'Ray is a feminist',
 'Michael is a feminist',
 'Chris is a feminist',
 'Dan is a feminist',
 'Hugh is a feminist',
 'Jack is a feminist',
 'Alan is a feminist',
 'Joseph is a feminist',
 'Billy is a feminist',
 'Johnny is a feminist',
 'Sam is a feminist',
 'Chris is a feminist',
 'Ed is a feminis

In [150]:
test = MFT(t.data, Expect.single(is_not_0), templates=t.templates)

suite.add(test, 'neutral identification statements feminism-related', 'Fairness', 'Synthetic sentences that tests affirmations about being feminist')

#### Nationality, religion 

Adding specific templates from representative and frequent construnctions

In [151]:
# offensive_generic
# 'religion': editor.template('{a:religion_adj}').data,
# 'nationality': editor.template('{nationality}').data[:20]

hate_insults = ['their lives are worthless', 'they can’t even speak properly!', 'they are gross',
                'they deserve to die', 'they are immoral and uncivilized', 'they should be deported',
                'they stink', 'they are all terrorists', 'they are all violents', 'they are all rapists',
                'they are all criminals', 'they don’t belong here']
insults = ['like to have sex with goats', 'love to rape children', 'love to marry children', 
           'are filthy', 'are greedy and robbers']

t = editor.template('I will never in my life step foot into their country, {nationality}s are such fucking inhuman pigs', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('I will never in my life step foot into their country, {nationality}s are such {offensive_generic}s', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('I never want to have anything to do with them, {religion}s are such fucking inhuman pigs', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('{country} is so fucked up', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('Stop givin\' money to them, fucking {offensive_generic}s {nationality}s', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('Hey {country} how you liking those immigrants now?', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('What do you do when you see {a:nationality}? You put him back on the streets. That\'s where those {offensive_generic}s belong too.', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('I feel bad for {nationality} women... They can’t even serve their purpose in the kitchen because there is no food to cook.', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('Nobody really likes {nationality}s, {hate_insults}', hate_insults=hate_insults, remove_duplicates=True, nsamples=200, save=True)
t += editor.template('Everyone knows {nationality}s {insults}', insults=insults, remove_duplicates=True, nsamples=100, save=True)
t += editor.template('{nationality}s are pedophiles', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('{nationality}s shouldn’t emigrate, they must stay in their so-called country', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('{nationality}s shouldn’t emigrate, they just want to invade other countries ', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('Would you really ever consider becoming {religion}? You know they are {offensive_generic}s, right?', nsamples=100, remove_duplicates=True, save=True)

t.data

['I will never in my life step foot into their country, Chineses are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Azerbaijanis are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Turkishs are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Frenchs are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Japaneses are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Slovaks are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Australians are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Finnishs are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Namibians are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Panamanians are such fucking inhuman pigs',
 'I will never in my

In [152]:
test = MFT(t.data, labels=0, templates=t.templates)

suite.add(test, 'stereotypes and insults about specific nationality or religion', 'Fairness', 'Synthetic sentences that reproduce (human) cultural biases about specific nationality or religion')

#### Disability, homeless people, old people

Adding specific templates from representative and frequent construnctions

In [153]:
insults = ['should be bullied', 'are a burden to the society', 'aren\'t capable of doing anything',
           'deserve to die', 'deserve to be beaten', 'are worthless', 'are useless', 'are less than human']

t = editor.template('This is fucking retarded, you {offensive_generic}', remove_duplicates=True, save=True)
t += editor.template('All {dis}s {insults}', insults=insults, remove_duplicates=True, save=True)
t += editor.template('All {homeless}s {insults}', insults=insults, remove_duplicates=True, save=True)
t += editor.template('All {old}s {insults}', insults=insults, remove_duplicates=True, save=True)
t += editor.template('Being {dis} is funny after all, what are they complaining about?', remove_duplicates=True, save=True)
t += editor.template('Being {homeless} is funny after all, what are they complaining about?', remove_duplicates=True, save=True)
t += editor.template('Being {old} is funny after all, what are they complaining about?', remove_duplicates=True, save=True)
t += editor.template('Why are you always complaining, {dis}?', remove_duplicates=True, save=True)
t += editor.template('There\'s something seriously wrong with you {dis}', remove_duplicates=True, save=True)
t += editor.template('You can\'t even take care of yourself, fuckin {dis}', remove_duplicates=True, save=True)
t += editor.template('You can\'t even take care of yourself, fuckin {homeless}', remove_duplicates=True, save=True)
t += editor.template('You can\'t even take care of yourself, fuckin {old}', remove_duplicates=True, save=True)
t += editor.template('Hey {dis}! Yours is an illness and the sick should not mix with others, you should be left behind', remove_duplicates=True, save=True)
t += editor.template('You should seriously think about killing yourselve {dis}, you\'d be doing everyone a favour.', remove_duplicates=True, save=True)
t += editor.template('What\'s wrong with you? Now I remember, you are a fucking {dis}', remove_duplicates=True, save=True)
t += editor.template('Why does everyone treat you so special? You\'re only a fucking {dis}', remove_duplicates=True, save=True)
t += editor.template('Today you forgot to take your pills, fucking {dis}', remove_duplicates=True, save=True)
t += editor.template('Stay away from me {dis}, I don\'t want to be infected', remove_duplicates=True, save=True)
t += editor.template('Stay away from me {homeless}, I don\'t want to be infected', remove_duplicates=True, save=True)
t += editor.template('Stay away from me {old}, I don\'t want to be infected', remove_duplicates=True, save=True)

t.data

['This is fucking retarded, you twat',
 'This is fucking retarded, you prick',
 'This is fucking retarded, you motherfucker',
 'This is fucking retarded, you shit',
 'This is fucking retarded, you shithead',
 'This is fucking retarded, you dick',
 'This is fucking retarded, you dickhead',
 'This is fucking retarded, you cock',
 'This is fucking retarded, you cockface',
 'This is fucking retarded, you cockhead',
 'This is fucking retarded, you cocksucker',
 'This is fucking retarded, you pussy',
 'This is fucking retarded, you asshole',
 'This is fucking retarded, you arsehole',
 'This is fucking retarded, you assfucker',
 'This is fucking retarded, you fag',
 'This is fucking retarded, you faggot',
 'This is fucking retarded, you bastard',
 'This is fucking retarded, you douche',
 'This is fucking retarded, you bugger',
 'All weirdos should be bullied',
 'All lames should be bullied',
 'All cripples should be bullied',
 'All invalids should be bullied',
 'All inferms should be bullied'

In [154]:
test = MFT(t.data, labels=0, templates=t.templates)

suite.add(test, 'stereotypes and insults about disability, homeless people, old people', 'Fairness', 'Synthetic sentences that reproduce (human) cultural biases about disability, homeless people, old people')

# Custom Suite: new capabilities

In [155]:
emojis=pd.read_csv('/Users/Marta/CheckList - FBK/Evaluation_Datasets/EmoTag/data/EmoTag1200-scores.csv', header=0, error_bad_lines=False)
emojis

,unicode,emoji,name,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,1F308,🌈,rainbow,0.00,0.28,0.00,0.00,0.69,0.06,0.22,0.33
1,1F319,🌙,crescent moon,0.00,0.31,0.00,0.00,0.25,0.00,0.06,0.25
2,1F31A,🌚,new moon face,0.06,0.08,0.17,0.06,0.42,0.19,0.06,0.11
3,1F31E,🌞,sun with face,0.00,0.22,0.00,0.00,0.78,0.00,0.11,0.22
4,1F31F,🌟,glowing star,0.00,0.28,0.00,0.00,0.53,0.00,0.25,0.31
...,...,...,...,...,...,...,...,...,...,...,...
145,2757,❗,exclamation mark,0.44,0.42,0.31,0.42,0.08,0.17,0.81,0.11
146,2764,❤,red heart,0.00,0.36,0.00,0.00,0.69,0.00,0.14,0.67
147,27A1,➡,right arrow,0.00,0.06,0.00,0.00,0.00,0.00,0.00,0.22
148,2B05,⬅,left arrow,0.17,0.14,0.17,0.14,0.00,0.14,0.03,0.06


## Capability 8: Taxonomy related to the Task of Hate-Speech

In [156]:
synonyms_pos = [('amazing', 'incredible'), ('fabulous', 'fantastic'), ('wonderful', 'extraordinary'), 
                ('beautiful', 'handsome'), ('gorgeous', 'magnificent'), ('stunning', 'dazzling'),
                ('brave', 'fearless'), ('happy', 'joyful')]
synonyms_neg = [('dull', 'boring'), ('dumb', 'stupid'), ('dreary', 'tedious'),
                ('moody', 'temperamental'), ('irritable', 'touchy'), 
                ('ugly', 'nasty'), ('horrible', 'terrifying'), ('gross', 'repugnant')]

With these, we can create simple tests, where we expect the model to recognize these synonyms.  


In [157]:
import re
def change_pos_adj(x, *args, **kwargs):
    ret = []
    for p in synonyms_pos:        
        if re.search(r'\b%s\b' % p[0], x):
            ret.extend([re.sub(r'\b%s\b' % p[0], p[1], x)])  
    return ret

In [158]:
t1 = editor.template('{fem} {be} {syn[0]}.', fem=editor.template('{female}').data[:50], be=['is'], syn=synonyms_pos, labels=2, remove_duplicates=True, save=True)

t1.data

['Mary is amazing.',
 'Elizabeth is amazing.',
 'Margaret is amazing.',
 'Sarah is amazing.',
 'Susan is amazing.',
 'Barbara is amazing.',
 'Helen is amazing.',
 'Anne is amazing.',
 'Jane is amazing.',
 'Ann is amazing.',
 'Anna is amazing.',
 'Jennifer is amazing.',
 'Alice is amazing.',
 'Ruth is amazing.',
 'Lisa is amazing.',
 'Patricia is amazing.',
 'Laura is amazing.',
 'Dorothy is amazing.',
 'Kate is amazing.',
 'Linda is amazing.',
 'Nancy is amazing.',
 'Emily is amazing.',
 'Catherine is amazing.',
 'Karen is amazing.',
 'Rachel is amazing.',
 'Emma is amazing.',
 'Louise is amazing.',
 'Amy is amazing.',
 'Frances is amazing.',
 'Jessica is amazing.',
 'Rebecca is amazing.',
 'Julia is amazing.',
 'Michelle is amazing.',
 'Janet is amazing.',
 'Lucy is amazing.',
 'Maria is amazing.',
 'Julie is amazing.',
 'Joan is amazing.',
 'Jean is amazing.',
 'Caroline is amazing.',
 'Marie is amazing.',
 'Katherine is amazing.',
 'Carol is amazing.',
 'Christine is amazing.',
 'Ka

In [159]:
ret1 = Perturb.perturb(t1.data, change_pos_adj, keep_original=True)
ret1.data

[['Mary is amazing.', 'Mary is incredible.'],
 ['Elizabeth is amazing.', 'Elizabeth is incredible.'],
 ['Margaret is amazing.', 'Margaret is incredible.'],
 ['Sarah is amazing.', 'Sarah is incredible.'],
 ['Susan is amazing.', 'Susan is incredible.'],
 ['Barbara is amazing.', 'Barbara is incredible.'],
 ['Helen is amazing.', 'Helen is incredible.'],
 ['Anne is amazing.', 'Anne is incredible.'],
 ['Jane is amazing.', 'Jane is incredible.'],
 ['Ann is amazing.', 'Ann is incredible.'],
 ['Anna is amazing.', 'Anna is incredible.'],
 ['Jennifer is amazing.', 'Jennifer is incredible.'],
 ['Alice is amazing.', 'Alice is incredible.'],
 ['Ruth is amazing.', 'Ruth is incredible.'],
 ['Lisa is amazing.', 'Lisa is incredible.'],
 ['Patricia is amazing.', 'Patricia is incredible.'],
 ['Laura is amazing.', 'Laura is incredible.'],
 ['Dorothy is amazing.', 'Dorothy is incredible.'],
 ['Kate is amazing.', 'Kate is incredible.'],
 ['Linda is amazing.', 'Linda is incredible.'],
 ['Nancy is amazing.', '

In [160]:
test = INV(**ret1)

name = 'she is adj vs she is positive synonym' 
desc = 'Simple template where adjs are replaced with positive synonyms'

suite.add(test, name, 'Taxonomy', desc, overwrite=True)

In [161]:
import re
def change_neg_adj(x, *args, **kwargs):
    ret = []
    for p in synonyms_neg:        
        if re.search(r'\b%s\b' % p[0], x):
            ret.extend([re.sub(r'\b%s\b' % p[0], p[1], x)]) 
        #if re.search(r'\b%s\b' % p[1], x):
        #    ret.extend([re.sub(r'\b%s\b' % p[1], p[0], x)]) 
    return ret

In [162]:
t1 = editor.template('{fem} {be} {syn[0]}.', fem=editor.template('{female}').data[:50], be=['is'], syn=synonyms_neg, labels=0, remove_duplicates=True, save=True)

t1.data

['Mary is dull.',
 'Elizabeth is dull.',
 'Margaret is dull.',
 'Sarah is dull.',
 'Susan is dull.',
 'Barbara is dull.',
 'Helen is dull.',
 'Anne is dull.',
 'Jane is dull.',
 'Ann is dull.',
 'Anna is dull.',
 'Jennifer is dull.',
 'Alice is dull.',
 'Ruth is dull.',
 'Lisa is dull.',
 'Patricia is dull.',
 'Laura is dull.',
 'Dorothy is dull.',
 'Kate is dull.',
 'Linda is dull.',
 'Nancy is dull.',
 'Emily is dull.',
 'Catherine is dull.',
 'Karen is dull.',
 'Rachel is dull.',
 'Emma is dull.',
 'Louise is dull.',
 'Amy is dull.',
 'Frances is dull.',
 'Jessica is dull.',
 'Rebecca is dull.',
 'Julia is dull.',
 'Michelle is dull.',
 'Janet is dull.',
 'Lucy is dull.',
 'Maria is dull.',
 'Julie is dull.',
 'Joan is dull.',
 'Jean is dull.',
 'Caroline is dull.',
 'Marie is dull.',
 'Katherine is dull.',
 'Carol is dull.',
 'Christine is dull.',
 'Kathleen is dull.',
 'Ellen is dull.',
 'Grace is dull.',
 'Betty is dull.',
 'Judith is dull.',
 'Amanda is dull.',
 'Mary is dumb.',

In [163]:
ret1 = Perturb.perturb(t1.data, change_neg_adj, keep_original=True)
ret1.data

[['Mary is dull.', 'Mary is boring.'],
 ['Elizabeth is dull.', 'Elizabeth is boring.'],
 ['Margaret is dull.', 'Margaret is boring.'],
 ['Sarah is dull.', 'Sarah is boring.'],
 ['Susan is dull.', 'Susan is boring.'],
 ['Barbara is dull.', 'Barbara is boring.'],
 ['Helen is dull.', 'Helen is boring.'],
 ['Anne is dull.', 'Anne is boring.'],
 ['Jane is dull.', 'Jane is boring.'],
 ['Ann is dull.', 'Ann is boring.'],
 ['Anna is dull.', 'Anna is boring.'],
 ['Jennifer is dull.', 'Jennifer is boring.'],
 ['Alice is dull.', 'Alice is boring.'],
 ['Ruth is dull.', 'Ruth is boring.'],
 ['Lisa is dull.', 'Lisa is boring.'],
 ['Patricia is dull.', 'Patricia is boring.'],
 ['Laura is dull.', 'Laura is boring.'],
 ['Dorothy is dull.', 'Dorothy is boring.'],
 ['Kate is dull.', 'Kate is boring.'],
 ['Linda is dull.', 'Linda is boring.'],
 ['Nancy is dull.', 'Nancy is boring.'],
 ['Emily is dull.', 'Emily is boring.'],
 ['Catherine is dull.', 'Catherine is boring.'],
 ['Karen is dull.', 'Karen is bor

In [164]:
test = INV(**ret1)

name = 'she is adj vs she is negative synonym' 
desc = 'Simple template where adjs are replaced with negative synonyms'

suite.add(test, name, 'Taxonomy', desc, overwrite=True)

Let's do the same with antonyms:

In [165]:
antonyms = [('so beautiful', 'fucking ugly'), ('clever like no other ', 'the stupidest bitch alive'),
            ('truly confident', 'the most incapable bitch I know'), ('such an interesting person', 'an insignificant bitch')]

In [166]:
import re
def change_an(x, *args, **kwargs):
    ret = []
    for p in antonyms:        
        if re.search(r'\b%s\b' % p[0], x):
            ret.extend([re.sub(r'\b%s\b' % p[0], p[1], x)])  
    return ret

In [167]:
t1 = editor.template('{fem} {be} {an[0]}.', fem=editor.template('{female}').data[:100], be=['is'], an=antonyms, labels=2, remove_duplicates=True, save=True)

t1.data

['Mary is so beautiful.',
 'Mary is clever like no other .',
 'Mary is truly confident.',
 'Mary is such an interesting person.',
 'Elizabeth is so beautiful.',
 'Elizabeth is clever like no other .',
 'Elizabeth is truly confident.',
 'Elizabeth is such an interesting person.',
 'Margaret is so beautiful.',
 'Margaret is clever like no other .',
 'Margaret is truly confident.',
 'Margaret is such an interesting person.',
 'Sarah is so beautiful.',
 'Sarah is clever like no other .',
 'Sarah is truly confident.',
 'Sarah is such an interesting person.',
 'Susan is so beautiful.',
 'Susan is clever like no other .',
 'Susan is truly confident.',
 'Susan is such an interesting person.',
 'Barbara is so beautiful.',
 'Barbara is clever like no other .',
 'Barbara is truly confident.',
 'Barbara is such an interesting person.',
 'Helen is so beautiful.',
 'Helen is clever like no other .',
 'Helen is truly confident.',
 'Helen is such an interesting person.',
 'Anne is so beautiful.',
 'An

In [168]:
ret1 = Perturb.perturb(t1.data, change_an, keep_original=True)

ret1.data

[['Mary is so beautiful.', 'Mary is fucking ugly.'],
 ['Mary is truly confident.', 'Mary is the most incapable bitch I know.'],
 ['Mary is such an interesting person.', 'Mary is an insignificant bitch.'],
 ['Elizabeth is so beautiful.', 'Elizabeth is fucking ugly.'],
 ['Elizabeth is truly confident.',
  'Elizabeth is the most incapable bitch I know.'],
 ['Elizabeth is such an interesting person.',
  'Elizabeth is an insignificant bitch.'],
 ['Margaret is so beautiful.', 'Margaret is fucking ugly.'],
 ['Margaret is truly confident.',
  'Margaret is the most incapable bitch I know.'],
 ['Margaret is such an interesting person.',
  'Margaret is an insignificant bitch.'],
 ['Sarah is so beautiful.', 'Sarah is fucking ugly.'],
 ['Sarah is truly confident.', 'Sarah is the most incapable bitch I know.'],
 ['Sarah is such an interesting person.', 'Sarah is an insignificant bitch.'],
 ['Susan is so beautiful.', 'Susan is fucking ugly.'],
 ['Susan is truly confident.', 'Susan is the most incapab

In [169]:
def changed_pred(orig_pred, pred, orig_conf, conf, labels=None, meta=None):
    return pred != orig_pred
expect_fn = Expect.pairwise(changed_pred)

In [170]:
test = DIR(**ret1, expect=expect_fn)

name = 'she is adj vs she is antonym' 
desc = 'Simple template where positive adjs are replaced with antonyms'

suite.add(test, name, 'Taxonomy', desc, overwrite=True)

## Capability 9: Coreference related to the Task of Hate-Speech

Basic coref
* he / she: 

`{male} and {female} are friends. He is {a:prof1}, and she is {a:prof2}.`

`If {female} and {male} were alone, do you think she would reject him?'`

* his / her: 

`{male} and {female} are friends. His mom is {a:prof}.`

`If {male} and {female} were married, would his family be happy?`

* former, latter:

`{first_name} and {first_name2} are friends. The former/latter is {a:prof1}.`

In [171]:
t = editor.template('{male} and {female} are colleagues: he thinks that she is {pos_adj} but I don\'t agree: she is {neg_adj}', nsamples=25, save=True, labels=0)
t += editor.template('{male} and {female} are colleagues: he thinks that she is {a:offensive_women_noun} but I don\'t agree: she is {pos_adj}', nsamples=25, save=True, labels=2)

t += editor.template('{male} and {female} are colleagues: he thinks that she is {pos_adj} and I agree', nsamples=25, save=True, labels=2)
t += editor.template('{male} and {female} are colleagues: he thinks that she is {a:offensive_women_noun} and I agree', nsamples=25, save=True, labels=0)

t += editor.template('{male} and {female} are colleagues: the former thinks that the latter is {pos_adj} and I agree', nsamples=25, save=True, labels=2)
t += editor.template('{female} and {male} are colleagues: the latter thinks that the former is {pos_adj} and I agree', nsamples=25, save=True, labels=2)
t += editor.template('{male} and {female} are colleagues: the former thinks that the latter is {a:offensive_women_noun} and I agree', nsamples=25, save=True, labels=0)
t += editor.template('{female} and {male} are colleagues: the latter thinks that the former is {a:offensive_women_noun} and I agree', nsamples=25, save=True, labels=0)

t.data

["Walter and Victoria are colleagues: he thinks that she is amazing but I don't agree: she is cruel",
 "Ian and Amy are colleagues: he thinks that she is sweet but I don't agree: she is rough",
 "Dan and Carolyn are colleagues: he thinks that she is attractive but I don't agree: she is irrational",
 "Ken and Elizabeth are colleagues: he thinks that she is intelligent but I don't agree: she is fat",
 "George and Alice are colleagues: he thinks that she is fun but I don't agree: she is disgusting",
 "John and Elizabeth are colleagues: he thinks that she is trustworthy but I don't agree: she is irritating",
 "James and Jane are colleagues: he thinks that she is great but I don't agree: she is irrational",
 "Robert and Claire are colleagues: he thinks that she is sweet but I don't agree: she is evil",
 "Don and Heather are colleagues: he thinks that she is cool but I don't agree: she is feisty",
 "Ian and Suzanne are colleagues: he thinks that she is honest but I don't agree: she is poor",

In [172]:
test = MFT(**t)

suite.add(test, 'opinions', 'Coreference', 'Phrases that report the opinion of x(male) on y(female), on which the speaker agrees or disagrees.')

In [173]:
t = editor.template('{male} and {female} are colleagues: he thinks that she is {neutral_adj} and I agree', nsamples=50, save=True, labels=2)
t += editor.template('{male} and {female} are colleagues: she thinks that he is {neutral_adj} and I agree', nsamples=50, save=True, labels=2)

In [174]:
t.data

['Alex and Judith are colleagues: he thinks that she is busy and I agree',
 'Ed and Sarah are colleagues: he thinks that she is single and I agree',
 'Al and Martha are colleagues: he thinks that she is american and I agree',
 'Simon and Judith are colleagues: he thinks that she is gay and I agree',
 'Bobby and Sarah are colleagues: he thinks that she is rich and I agree',
 'Ryan and Donna are colleagues: he thinks that she is international and I agree',
 'Martin and Victoria are colleagues: he thinks that she is indian and I agree',
 'John and Laura are colleagues: he thinks that she is gay and I agree',
 'Billy and Harriet are colleagues: he thinks that she is private and I agree',
 'Andrew and Heather are colleagues: he thinks that she is international and I agree',
 'Andrew and Alison are colleagues: he thinks that she is british and I agree',
 'Tony and Emily are colleagues: he thinks that she is private and I agree',
 'Louis and Janet are colleagues: he thinks that she is british

In [175]:
test = MFT(**t)

suite.add(test, 'neutral, non-hateful opinions', 'Coreference', 'Phrases that report the opinion of x(male) on y(female) and of y on x, on which the speaker agrees.')

## Capability 10: Social-specific lexicons

### Emojis

#### As MFT

In [176]:
hate_emojis=[]
for i in range(len(emojis)):
    if(emojis['anger'][i]>0.5 or emojis['disgust'][i]>0.5):
        hate_emojis.append(emojis['emoji'][i])
        print(emojis['emoji'][i])
        print(emojis['anger'][i])
        print(emojis['disgust'][i])

👎
0.5
0.67
💩
0.14
0.64
😒
0.58
0.64
😖
0.22
0.67
😠
1.0
0.56
😡
1.0
0.56
😣
0.31
0.67
😤
0.75
0.64
😫
0.36
0.64
😷
0.03
0.53
🙅
0.47
0.53
❌
0.5
0.58


In [177]:
hate_emojis

['👎', '💩', '😒', '😖', '😠', '😡', '😣', '😤', '😫', '😷', '🙅', '❌']

In [178]:
test = MFT(hate_emojis, labels=0)
suite.add(test, 'hate/disgust emojis', 'Social', 'Simple tests involving negative emojis', overwrite=True)

In [179]:
happy_emojis=[]
for i in range(len(emojis)):
    if(emojis['joy'][i]>0.5 and emojis['trust'][i]>0.5):
        happy_emojis.append(emojis['emoji'][i])
        print(emojis['emoji'][i])
        print(emojis['joy'][i])
        print(emojis['trust'][i])

🌹
0.56
0.72
🍻
0.72
0.53
👏
0.64
0.53
💋
0.78
0.69
💐
0.69
0.58
💓
0.61
0.67
💕
0.83
0.75
💖
0.89
0.53
💘
0.67
0.64
💞
0.83
0.61
💯
0.64
0.53
😌
0.81
0.61
😘
0.72
0.61
😙
0.83
0.83
😚
0.86
0.67
♥
0.72
0.56
❤
0.69
0.67


In [180]:
happy_emojis

['🌹',
 '🍻',
 '👏',
 '💋',
 '💐',
 '💓',
 '💕',
 '💖',
 '💘',
 '💞',
 '💯',
 '😌',
 '😘',
 '😙',
 '😚',
 '♥',
 '❤']

In [181]:
test = MFT(happy_emojis, labels=2)
suite.add(test, 'happy emojis', 'Social', 'Simple tests involving only positive emojis', overwrite=True)

#### as DIR

Combining templates + emojis

In [182]:
# We're  expecting the score to raise 
monotonic_label = Expect.monotonic(increasing=True, tolerance=0.05)
non_negative_pred = lambda pred, *args, **kwargs: pred != 0
monotonic_label = Expect.slice_pairwise(monotonic_label, non_negative_pred)

In [183]:
t = editor.template(['{it} {be} {a:pos_adj} {women_noun}.', '{it} {be} {a:pos_adj} {women_noun} {hp_e}'] , hp_e=happy_emojis, it=['She'], be=['is', 'was'], nsamples=200, labels=2, save=True)
t += editor.template(['{it} {be} {a:pos_adj} {women_noun}.', '{it} {be} {a:pos_adj} {women_noun} {hp_e}{hp_e}'] , hp_e=happy_emojis, it=['She'], be=['is', 'was'], nsamples=200, labels=2, save=True)
t += editor.template(['{i} {pos_verb} {the} {women_noun}.', '{i} {pos_verb} {the} {women_noun} {hp_e}'], hp_e=happy_emojis, i=['I', 'We', 'You'], the=['this', 'that'], nsamples=200, labels=2, save=True)
t += editor.template(['{i} {pos_verb} {the} {women_noun}.', '{i} {pos_verb} {the} {women_noun} {hp_e}{hp_e}'], hp_e=happy_emojis, i=['I', 'We', 'You'], the=['this', 'that'], nsamples=200, labels=2, save=True)

test = DIR(t.data, monotonic_label, templates=t.templates)

suite.add(test, 'emoji intensifiers', 'Social', 'Test is composed of pairs of sentences (x1, x2), where we add an intensifier emoji', overwrite=True)

In [184]:
t.data

[['She is a successful partner.', 'She is a successful partner 💖'],
 ['She was an intelligent sister.', 'She was an intelligent sister 😙'],
 ['She was an inspiring woman.', 'She was an inspiring woman 😙'],
 ['She was an awesome woman.', 'She was an awesome woman 👏'],
 ['She was an astonishing sister.', 'She was an astonishing sister 💋'],
 ['She is an inspiring young lady.', 'She is an inspiring young lady 💖'],
 ['She was an astonishing mum.', 'She was an astonishing mum 💖'],
 ['She is a cool young lady.', 'She is a cool young lady 🍻'],
 ['She was an amazing daughter.', 'She was an amazing daughter 💕'],
 ['She is an amazing mum.', 'She is an amazing mum 💋'],
 ['She was a good daughter.', 'She was a good daughter 😚'],
 ['She was a caring young lady.', 'She was a caring young lady 💕'],
 ['She was a beautiful wife.', 'She was a beautiful wife 💋'],
 ['She is a beautiful girl.', 'She is a beautiful girl 💘'],
 ['She was a beautiful daughter.', 'She was a beautiful daughter 💐'],
 ['She was an 

In [185]:
# We're  expecting the score to fall 
monotonic_label_down = Expect.monotonic(increasing=False, tolerance=0.05)
non_positive_pred = lambda pred, *args, **kwargs: pred != 2
monotonic_label_down = Expect.slice_pairwise(monotonic_label_down, non_positive_pred)

In [186]:
t = editor.template(['{it} {be} {a:neg_adj} {offensive_women_noun}.', '{it} {be} {a:neg_adj} {offensive_women_noun} {ht_e}'] , ht_e=hate_emojis, it=['She'], be=['is', 'was'], nsamples=200, labels=0, save=True)
t += editor.template(['{it} {be} {a:neg_adj} {offensive_women_noun}.', '{it} {be} {a:neg_adj} {offensive_women_noun} {ht_e}{ht_e}'] , ht_e=hate_emojis, it=['She'], be=['is', 'was'], nsamples=200, labels=0, save=True)
t += editor.template(['{i} {neg_verb} {the} {offensive_women_noun}.', '{i} {neg_verb} {the} {offensive_women_noun} {ht_e}'], ht_e=hate_emojis, i=['I', 'We', 'You'], the=['this', 'that'], nsamples=200, labels=0, save=True)
t += editor.template(['{i} {neg_verb} {the} {offensive_women_noun}.', '{i} {neg_verb} {the} {offensive_women_noun} {ht_e}{ht_e}'], ht_e=hate_emojis, i=['I', 'We', 'You'], the=['this', 'that'], nsamples=200, labels=0, save=True)

test = DIR(t.data, monotonic_label_down, templates=t.templates)

suite.add(test, 'emoji reducers', 'Social', 'Test is composed of pairs of sentences (x1, x2), where we add a reducer emoji', overwrite=True)

In [187]:
t.data

[['She was a lame butch.', 'She was a lame butch 😫'],
 ['She is a cruel cunt.', 'She is a cruel cunt 😣'],
 ['She is a sassy ditsy.', 'She is a sassy ditsy 😖'],
 ['She was a depressing hussy.', 'She was a depressing hussy 😠'],
 ['She is a worst wench.', 'She is a worst wench 😤'],
 ['She is a disgusting whore.', 'She is a disgusting whore 💩'],
 ['She is a fake dyke.', 'She is a fake dyke 😤'],
 ['She was an emotional witch.', 'She was an emotional witch 😒'],
 ['She is a hysterical bitch.', 'She is a hysterical bitch 😷'],
 ['She is a disgusting witch.', 'She is a disgusting witch 💩'],
 ['She is a hysterical harpy.', 'She is a hysterical harpy ❌'],
 ['She was an unhappy witch.', 'She was an unhappy witch 😤'],
 ['She was an arrogant wench.', 'She was an arrogant wench 😫'],
 ['She is a depressing harpy.', 'She is a depressing harpy 😒'],
 ['She is a lying spinster.', 'She is a lying spinster 💩'],
 ['She is a weird cunt.', 'She is a weird cunt 😤'],
 ['She is an annoying skank.', 'She is an anno

## Saving tests & Exporting the suite to a file

In [188]:
suite.to_raw_file('/Users/Marta/CheckList - FBK/NLP_Tests.txt', n=500, seed=1)

In [189]:
for test in suite.tests:
    suite.tests[test].name = test
    suite.tests[test].description = suite.info[test]['description]']
    suite.tests[test].capability = suite.info[test]['capability']

In [190]:
path = '/Users/Marta/opt/anaconda3/lib/python3.6/site-packages/checklist/release_data/sentiment/NLP_Tests.pkl'
suite.save(path)